In [8]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector as connection
import pymysql
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
from IPython.display import display
import calendar
from matplotlib.ticker import FuncFormatter
import json

def get_db_connection():
    return pymysql.connect(**db_config)
# Load environment variables
load_dotenv()

# Database configuration with enhanced security
db_config = {
    "host": "localhost",
    "port": 3306,
    "user": "user",
    "password": "Iron_Men",
    "database": "hospital",
    "charset": "utf8mb4",
    "cursorclass": pymysql.cursors.DictCursor
}

# Initialize database connection
def init_db():
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # Enhanced Patients table with foreign key to insurance_provider
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS patient (
                patient_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL,
                age INT NOT NULL,
                gender ENUM('Male', 'Female', 'Other') NOT NULL,
                blood_type ENUM('A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'),
                address VARCHAR(200),
                phone VARCHAR(15),
                email VARCHAR(100),
                insurance_provider_id INT,
                insurance_policy_number VARCHAR(50),
                primary_physician VARCHAR(100),
                emergency_contact VARCHAR(100),
                emergency_phone VARCHAR(15),
                medical_history TEXT,
                current_medications TEXT,
                allergies TEXT,
                disease VARCHAR(100),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                INDEX idx_patient_name (name),
                INDEX idx_patient_disease (disease),
                CONSTRAINT fk_patient_insurance FOREIGN KEY (insurance_provider_id) 
                    REFERENCES insurance_provider(provider_id) ON DELETE SET NULL
            );
            """)

            # Enhanced Doctors table with foreign key to department
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS doctor (
                doctor_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL,
                specialization VARCHAR(100) NOT NULL,
                department_id INT,
                qualification VARCHAR(100),
                years_of_experience INT,
                phone VARCHAR(15),
                email VARCHAR(100),
                consultation_fee DECIMAL(10,2) NOT NULL,
                availability TEXT COMMENT 'JSON string of available days/times',
                bio TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                INDEX idx_doctor_specialization (specialization),
                INDEX idx_doctor_department (department_id),
                CONSTRAINT fk_doctor_department FOREIGN KEY (department_id) 
                    REFERENCES department(department_id) ON DELETE SET NULL
            );
            """)

            # Rest of your table creation statements...
            # (Keep all the other CREATE TABLE statements as they are, 
            # just remove the ALTER TABLE statements at the end)
            
            # New: Department table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS department (
                department_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL UNIQUE,
                head_of_department VARCHAR(100),
                phone VARCHAR(15),
                email VARCHAR(100),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
            );
            """)

            # New: Staff table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS staff (
                staff_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL,
                role VARCHAR(50) NOT NULL,
                department_id INT,
                phone VARCHAR(15),
                email VARCHAR(100),
                address VARCHAR(200),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                FOREIGN KEY (department_id) REFERENCES department(department_id) ON DELETE SET NULL
            );
            """)

            # New: Insurance Provider table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS insurance_provider (
                provider_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL UNIQUE,
                contact_person VARCHAR(100),
                phone VARCHAR(15),
                email VARCHAR(100),
                address VARCHAR(200),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
            );
            """)

            # New: Test Type table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS test_type (
                test_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL UNIQUE,
                description TEXT,
                cost DECIMAL(10, 2) NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
            );
            """)

            # New: Patient Test table (to link patients to tests)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS patient_test (
                patient_test_id INT AUTO_INCREMENT PRIMARY KEY,
                patient_id INT NOT NULL,
                doctor_id INT,
                test_id INT NOT NULL,
                date_ordered DATE NOT NULL,
                date_completed DATE,
                results TEXT,
                status ENUM('Ordered', 'InProgress', 'Completed', 'Cancelled') DEFAULT 'Ordered',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                FOREIGN KEY (patient_id) REFERENCES patient(patient_id) ON DELETE CASCADE,
                FOREIGN KEY (doctor_id) REFERENCES doctor(doctor_id) ON DELETE SET NULL,
                FOREIGN KEY (test_id) REFERENCES test_type(test_id) ON DELETE CASCADE
            );
            """)
            
            # Enhanced Appointments table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS appointment (
                appointment_id INT AUTO_INCREMENT PRIMARY KEY,
                patient_id INT NOT NULL,
                doctor_id INT NOT NULL,
                date DATE NOT NULL,
                time TIME NOT NULL,
                duration INT DEFAULT 30 COMMENT 'Duration in minutes',
                reason TEXT,
                status ENUM('Scheduled', 'Completed', 'Cancelled', 'No-Show') DEFAULT 'Scheduled',
                notes TEXT,
                follow_up_date DATE,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                FOREIGN KEY (patient_id) REFERENCES patient(patient_id) ON DELETE CASCADE,
                FOREIGN KEY (doctor_id) REFERENCES doctor(doctor_id) ON DELETE CASCADE,
                INDEX idx_appointment_date (date),
                INDEX idx_appointment_status (status)
            );
            """)

            # Enhanced Billing table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS billing (
                bill_id INT AUTO_INCREMENT PRIMARY KEY,
                patient_id INT NOT NULL,
                doctor_id INT NOT NULL,
                appointment_id INT,
                invoice_number VARCHAR(20) UNIQUE,
                amount DECIMAL(10,2) NOT NULL,
                tax DECIMAL(10,2) DEFAULT 0.00,
                discount DECIMAL(10,2) DEFAULT 0.00,
                total_amount DECIMAL(10,2) GENERATED ALWAYS AS (amount + tax - discount) STORED,
                date DATE NOT NULL,
                due_date DATE NOT NULL,
                status ENUM('Paid', 'Unpaid', 'Partial', 'Overdue') DEFAULT 'Unpaid',
                payment_method ENUM('Cash', 'Credit Card', 'Debit Card', 'Insurance', 'Bank Transfer', 'Other'),
                payment_details TEXT,
                items TEXT COMMENT 'JSON string of billed items',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                FOREIGN KEY (patient_id) REFERENCES patient(patient_id) ON DELETE CASCADE,
                FOREIGN KEY (doctor_id) REFERENCES doctor(doctor_id) ON DELETE CASCADE,
                FOREIGN KEY (appointment_id) REFERENCES appointment(appointment_id) ON DELETE SET NULL,
                INDEX idx_billing_status (status),
                INDEX idx_billing_date (date)
            );
            """)

            # Enhanced Medical records table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS medical_record (
                record_id INT AUTO_INCREMENT PRIMARY KEY,
                patient_id INT NOT NULL,
                doctor_id INT,
                visit_type ENUM('Routine Checkup', 'Emergency', 'Follow-up', 'Specialist Consultation', 'Other'),
                diagnosis TEXT,
                symptoms TEXT,
                treatment TEXT,
                prescription TEXT,
                tests_ordered TEXT,
                test_results TEXT,
                notes TEXT,
                follow_up_required BOOLEAN DEFAULT FALSE,
                follow_up_date DATE,
                date DATE NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                FOREIGN KEY (patient_id) REFERENCES patient(patient_id) ON DELETE CASCADE,
                FOREIGN KEY (doctor_id) REFERENCES doctor(doctor_id) ON DELETE SET NULL,
                INDEX idx_record_date (date),
                INDEX idx_record_visit_type (visit_type)
            );
            """)

            # Enhanced Users table for authentication
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                user_id INT AUTO_INCREMENT PRIMARY KEY,
                username VARCHAR(50) NOT NULL UNIQUE,
                password_hash VARCHAR(255) NOT NULL,
                salt VARCHAR(100) NOT NULL,
                role ENUM('Admin', 'Doctor', 'Staff', 'Patient') NOT NULL,
                related_id INT COMMENT 'ID of related record in doctor/patient/staff table',
                last_login DATETIME,
                login_attempts INT DEFAULT 0,
                account_locked BOOLEAN DEFAULT FALSE,
                password_changed_at DATETIME,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                INDEX idx_user_role (role)
            );
            """)

            # Inventory table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS inventory (
                item_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL,
                category VARCHAR(100) NOT NULL,
                description TEXT,
                quantity INT NOT NULL,
                unit VARCHAR(20) NOT NULL,
                price DECIMAL(10,2) NOT NULL,
                supplier VARCHAR(100),
                expiry_date DATE,
                threshold INT COMMENT 'Minimum quantity before reorder',
                last_restocked DATE,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                INDEX idx_inventory_category (category),
                INDEX idx_inventory_expiry (expiry_date)
            );
            """)

            # Add foreign key constraints after all tables are created
            cursor.execute("ALTER TABLE patient ADD CONSTRAINT fk_patient_insurance FOREIGN KEY (insurance_provider_id) REFERENCES insurance_provider(provider_id) ON DELETE SET NULL;")
            cursor.execute("ALTER TABLE doctor ADD CONSTRAINT fk_doctor_department FOREIGN KEY (department_id) REFERENCES department(department_id) ON DELETE SET NULL;")
            
            connection.commit()
            print("Database tables initialized successfully with enhanced schema.")
    except Exception as e:
        print(f"Error initializing database: {e}")
    finally:
        connection.close()

# --- New: Department Management Functions ---
def add_department(name, head_of_department=None, phone=None, email=None):
    """Add a new department to the database."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO department (name, head_of_department, phone, email) VALUES (%s, %s, %s, %s)"
            cursor.execute(sql, (name, head_of_department, phone, email))
        connection.commit()
        print("Department added successfully!")
    except Exception as e:
        print(f"Error adding department: {e}")
    finally:
        connection.close()

def view_departments():
    """View all departments."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "SELECT department_id, name, head_of_department, phone, email FROM department ORDER BY name"
            cursor.execute(sql)
            departments = cursor.fetchall()
            df = pd.DataFrame(departments)
            if df.empty:
                print("No departments found.")
                return None
            display(df)
            return df
    except Exception as e:
        print(f"Error retrieving departments: {e}")
    finally:
        connection.close()
        
def update_department(department_id, name=None, head_of_department=None, phone=None, email=None):
    """Update an existing department's information."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            updates = {}
            if name is not None: updates['name'] = name
            if head_of_department is not None: updates['head_of_department'] = head_of_department
            if phone is not None: updates['phone'] = phone
            if email is not None: updates['email'] = email
            
            if not updates:
                print("No updates provided.")
                return

            set_clause = ', '.join([f"{key} = %s" for key in updates.keys()])
            sql = f"UPDATE department SET {set_clause} WHERE department_id = %s"
            values = list(updates.values()) + [department_id]
            
            cursor.execute(sql, values)
            connection.commit()
            print("Department updated successfully!")
    except Exception as e:
        print(f"Error updating department: {e}")
    finally:
        connection.close()

def delete_department(department_id):
    """Delete a department by ID."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "DELETE FROM department WHERE department_id = %s"
            cursor.execute(sql, (department_id,))
            connection.commit()
            if cursor.rowcount > 0:
                print("Department deleted successfully!")
            else:
                print("Department not found.")
    except Exception as e:
        print(f"Error deleting department: {e}")
    finally:
        connection.close()

# --- New: Staff Management Functions ---
def add_staff(name, role, department_id, phone=None, email=None, address=None):
    """Add a new staff member."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO staff (name, role, department_id, phone, email, address) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (name, role, department_id, phone, email, address))
            connection.commit()
            print("Staff member added successfully!")
    except Exception as e:
        print(f"Error adding staff member: {e}")
    finally:
        connection.close()

def view_staff():
    """View all staff members with department names."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT s.staff_id, s.name, s.role, d.name AS department, s.phone, s.email
            FROM staff s
            LEFT JOIN department d ON s.department_id = d.department_id
            ORDER BY s.name
            """
            cursor.execute(sql)
            staff = cursor.fetchall()
            df = pd.DataFrame(staff)
            if df.empty:
                print("No staff records found.")
                return None
            display(df)
            return df
    except Exception as e:
        print(f"Error retrieving staff records: {e}")
    finally:
        connection.close()

def update_staff(staff_id, name=None, role=None, department_id=None, phone=None, email=None, address=None):
    """Update a staff member's information."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            updates = {}
            if name is not None: updates['name'] = name
            if role is not None: updates['role'] = role
            if department_id is not None: updates['department_id'] = department_id
            if phone is not None: updates['phone'] = phone
            if email is not None: updates['email'] = email
            if address is not None: updates['address'] = address

            if not updates:
                print("No updates provided.")
                return

            set_clause = ', '.join([f"{key} = %s" for key in updates.keys()])
            sql = f"UPDATE staff SET {set_clause} WHERE staff_id = %s"
            values = list(updates.values()) + [staff_id]

            cursor.execute(sql, values)
            connection.commit()
            print("Staff member updated successfully!")
    except Exception as e:
        print(f"Error updating staff member: {e}")
    finally:
        connection.close()

def delete_staff(staff_id):
    """Delete a staff member by ID."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "DELETE FROM staff WHERE staff_id = %s"
            cursor.execute(sql, (staff_id,))
            connection.commit()
            if cursor.rowcount > 0:
                print("Staff member deleted successfully!")
            else:
                print("Staff member not found.")
    except Exception as e:
        print(f"Error deleting staff member: {e}")
    finally:
        connection.close()

# --- New: Insurance Provider Management Functions ---
def add_insurance_provider(name, contact_person=None, phone=None, email=None, address=None):
    """Add a new insurance provider."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO insurance_provider (name, contact_person, phone, email, address) VALUES (%s, %s, %s, %s, %s)"
            cursor.execute(sql, (name, contact_person, phone, email, address))
            connection.commit()
            print("Insurance provider added successfully!")
    except Exception as e:
        print(f"Error adding insurance provider: {e}")
    finally:
        connection.close()

def view_insurance_providers():
    """View all insurance providers."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "SELECT provider_id, name, contact_person, phone, email, address FROM insurance_provider ORDER BY name"
            cursor.execute(sql)
            providers = cursor.fetchall()
            df = pd.DataFrame(providers)
            if df.empty:
                print("No insurance providers found.")
                return None
            display(df)
            return df
    except Exception as e:
        print(f"Error retrieving insurance providers: {e}")
    finally:
        connection.close()

def update_insurance_provider(provider_id, name=None, contact_person=None, phone=None, email=None, address=None):
    """Update an insurance provider's information."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            updates = {}
            if name is not None: updates['name'] = name
            if contact_person is not None: updates['contact_person'] = contact_person
            if phone is not None: updates['phone'] = phone
            if email is not None: updates['email'] = email
            if address is not None: updates['address'] = address

            if not updates:
                print("No updates provided.")
                return

            set_clause = ', '.join([f"{key} = %s" for key in updates.keys()])
            sql = f"UPDATE insurance_provider SET {set_clause} WHERE provider_id = %s"
            values = list(updates.values()) + [provider_id]

            cursor.execute(sql, values)
            connection.commit()
            print("Insurance provider updated successfully!")
    except Exception as e:
        print(f"Error updating insurance provider: {e}")
    finally:
        connection.close()

def delete_insurance_provider(provider_id):
    """Delete an insurance provider by ID."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "DELETE FROM insurance_provider WHERE provider_id = %s"
            cursor.execute(sql, (provider_id,))
            connection.commit()
            if cursor.rowcount > 0:
                print("Insurance provider deleted successfully!")
            else:
                print("Insurance provider not found.")
    except Exception as e:
        print(f"Error deleting insurance provider: {e}")
    finally:
        connection.close()

# --- New: Test Management Functions ---
def add_test_type(name, cost, description=None):
    """Add a new medical test type."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO test_type (name, cost, description) VALUES (%s, %s, %s)"
            cursor.execute(sql, (name, cost, description))
            connection.commit()
            print("Medical test type added successfully!")
    except Exception as e:
        print(f"Error adding medical test type: {e}")
    finally:
        connection.close()

def add_patient_test(patient_id, test_id, doctor_id, date_ordered=None):
    """Order a new test for a patient."""
    if date_ordered is None:
        date_ordered = datetime.now().date()
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO patient_test (patient_id, doctor_id, test_id, date_ordered) VALUES (%s, %s, %s, %s)"
            cursor.execute(sql, (patient_id, doctor_id, test_id, date_ordered))
            connection.commit()
            print("Patient test ordered successfully!")
    except Exception as e:
        print(f"Error ordering patient test: {e}")
    finally:
        connection.close()

def update_patient_test(patient_test_id, date_completed=None, results=None, status=None):
    """Update a patient's test record."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            updates = {}
            if date_completed is not None: updates['date_completed'] = date_completed
            if results is not None: updates['results'] = results
            if status is not None: updates['status'] = status
            
            if not updates:
                print("No updates provided.")
                return

            set_clause = ', '.join([f"{key} = %s" for key in updates.keys()])
            sql = f"UPDATE patient_test SET {set_clause} WHERE patient_test_id = %s"
            values = list(updates.values()) + [patient_test_id]
            
            cursor.execute(sql, values)
            connection.commit()
            print("Patient test updated successfully!")
    except Exception as e:
        print(f"Error updating patient test: {e}")
    finally:
        connection.close()

def view_patient_tests():
    """View all patient test records."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT pt.patient_test_id, p.name AS patient_name, d.name AS doctor_name, tt.name AS test_name,
                   pt.date_ordered, pt.date_completed, pt.status, pt.results
            FROM patient_test pt
            JOIN patient p ON pt.patient_id = p.patient_id
            JOIN doctor d ON pt.doctor_id = d.doctor_id
            JOIN test_type tt ON pt.test_id = tt.test_id
            ORDER BY pt.date_ordered DESC
            """
            cursor.execute(sql)
            tests = cursor.fetchall()
            df = pd.DataFrame(tests)
            if df.empty:
                print("No patient test records found.")
                return None
            display(df)
            return df
    except Exception as e:
        print(f"Error retrieving patient test records: {e}")
    finally:
        connection.close()

# --- New: Inventory Management Functions ---
def add_inventory_item(name, category, quantity, unit, price, supplier=None, expiry_date=None, threshold=None, last_restocked=None, description=None):
    """Add a new item to the inventory."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO inventory (name, category, quantity, unit, price, supplier, expiry_date, threshold, last_restocked, description)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (name, category, quantity, unit, price, supplier, expiry_date, threshold, last_restocked, description))
            connection.commit()
            print("Inventory item added successfully!")
    except Exception as e:
        print(f"Error adding inventory item: {e}")
    finally:
        connection.close()

def update_inventory_item(item_id, quantity=None, price=None, threshold=None, supplier=None, last_restocked=None, expiry_date=None, name=None, category=None, unit=None, description=None):
    """Update details of an existing inventory item."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            updates = {}
            if quantity is not None: updates['quantity'] = quantity
            if price is not None: updates['price'] = price
            if threshold is not None: updates['threshold'] = threshold
            if supplier is not None: updates['supplier'] = supplier
            if last_restocked is not None: updates['last_restocked'] = last_restocked
            if expiry_date is not None: updates['expiry_date'] = expiry_date
            if name is not None: updates['name'] = name
            if category is not None: updates['category'] = category
            if unit is not None: updates['unit'] = unit
            if description is not None: updates['description'] = description

            if not updates:
                print("No updates provided.")
                return

            set_clause = ', '.join([f"{key} = %s" for key in updates.keys()])
            sql = f"UPDATE inventory SET {set_clause} WHERE item_id = %s"
            values = list(updates.values()) + [item_id]
            
            cursor.execute(sql, values)
            connection.commit()
            print("Inventory item updated successfully!")
    except Exception as e:
        print(f"Error updating inventory item: {e}")
    finally:
        connection.close()

def delete_inventory_item(item_id):
    """Delete an item from the inventory by ID."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "DELETE FROM inventory WHERE item_id = %s"
            cursor.execute(sql, (item_id,))
            connection.commit()
            if cursor.rowcount > 0:
                print("Inventory item deleted successfully!")
            else:
                print("Inventory item not found.")
    except Exception as e:
        print(f"Error deleting inventory item: {e}")
    finally:
        connection.close()

def view_inventory():
    """View all inventory items and generate a stock report."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "SELECT item_id, name, category, quantity, unit, price, supplier, expiry_date, threshold FROM inventory ORDER BY name"
            cursor.execute(sql)
            items = cursor.fetchall()
            df = pd.DataFrame(items)
            
            if df.empty:
                print("No inventory items found.")
                return None
            
            print("\n=== CURRENT INVENTORY ===")
            display(df)
            
            # Low stock report
            low_stock_df = df[df['quantity'] <= df['threshold']]
            if not low_stock_df.empty:
                print("\n--- LOW STOCK ALERT ---")
                display(low_stock_df)
            else:
                print("\nAll items are adequately stocked.")
                
            # Category distribution chart
            plt.figure(figsize=(10, 6))
            category_counts = df['category'].value_counts()
            category_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
            plt.title('Inventory Distribution by Category')
            plt.ylabel('')
            plt.tight_layout()
            plt.show()

            return df
    except Exception as e:
        print(f"Error retrieving inventory: {e}")
    finally:
        connection.close()

def check_low_stock():
    """Checks for and prints a report of all low-stock items."""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = "SELECT item_id, name, quantity, threshold, unit, supplier FROM inventory WHERE quantity <= threshold"
            cursor.execute(sql)
            low_stock_items = cursor.fetchall()
            
            if low_stock_items:
                print("\n--- ATTENTION: LOW STOCK ITEMS ---")
                df = pd.DataFrame(low_stock_items)
                display(df)
            else:
                print("\nNo items are currently below their reorder threshold.")
                
    except Exception as e:
        print(f"Error checking low stock: {e}")
    finally:
        connection.close()

# --- Core Functions (kept the same, with minor adjustments for new schema) ---

def add_patient(name, age, gender, blood_type=None, address=None, phone=None, email=None, 
                insurance_provider_id=None, insurance_policy_number=None, primary_physician=None,
                emergency_contact=None, emergency_phone=None, medical_history=None,
                current_medications=None, allergies=None, disease=None):
    """Add a new patient to the database with enhanced information"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO patient (
                name, age, gender, blood_type, address, phone, email, 
                insurance_provider_id, insurance_policy_number, primary_physician,
                emergency_contact, emergency_phone, medical_history,
                current_medications, allergies, disease
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (
                name, age, gender, blood_type, address, phone, email,
                insurance_provider_id, insurance_policy_number, primary_physician,
                emergency_contact, emergency_phone, medical_history,
                current_medications, allergies, disease
            ))
        connection.commit()
        print("Patient added successfully with comprehensive details!")
    except Exception as e:
        print(f"Error adding patient: {e}")
    finally:
        connection.close()

def add_doctor(name, specialization, consultation_fee, department_id=None, qualification=None,
               years_of_experience=None, phone=None, email=None, availability=None, bio=None):
    """Add a new doctor to the database with enhanced information"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO doctor (
                name, specialization, department_id, qualification, years_of_experience,
                phone, email, consultation_fee, availability, bio
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (
                name, specialization, department_id, qualification, years_of_experience,
                phone, email, consultation_fee, availability, bio
            ))
        connection.commit()
        print("Doctor added successfully with comprehensive details!")
    except Exception as e:
        print(f"Error adding doctor: {e}")
    finally:
        connection.close()

def schedule_appointment(patient_id, doctor_id, date, time, reason=None, duration=30, notes=None):
    """Schedule a new appointment with enhanced details"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO appointment (
                patient_id, doctor_id, date, time, duration, reason, notes
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (patient_id, doctor_id, date, time, duration, reason, notes))
        connection.commit()
        print("Appointment scheduled successfully with duration and reason!")
    except Exception as e:
        print(f"Error scheduling appointment: {e}")
    finally:
        connection.close()

def update_appointment_status(appointment_id, status, follow_up_date=None):
    """Update appointment status with optional follow-up date"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            if follow_up_date:
                sql = """
                UPDATE appointment 
                SET status = %s, follow_up_date = %s
                WHERE appointment_id = %s
                """
                cursor.execute(sql, (status, follow_up_date, appointment_id))
            else:
                sql = """
                UPDATE appointment 
                SET status = %s 
                WHERE appointment_id = %s
                """
                cursor.execute(sql, (status, appointment_id))
        connection.commit()
        print("Appointment status updated successfully!")
    except Exception as e:
        print(f"Error updating appointment status: {e}")
    finally:
        connection.close()

def generate_bill(patient_id, doctor_id, amount, date, items=None,
                  status='Unpaid', appointment_id=None, payment_method=None,
                  tax=0.00, discount=0.00, due_date=None):
    """Generate a new bill with enhanced details"""
    if due_date is None:
        due_date = (datetime.strptime(date, '%Y-%m-%d') + timedelta(days=30)).strftime('%Y-%m-%d')
    
    # Generate invoice number
    invoice_number = f"INV-{datetime.now().strftime('%Y%m%d')}-{patient_id:04d}"
    
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO billing (
                patient_id, doctor_id, appointment_id, invoice_number,
                amount, tax, discount, date, due_date, 
                status, payment_method, items
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (
                patient_id, doctor_id, appointment_id, invoice_number,
                amount, tax, discount, date, due_date,
                status, payment_method, items
            ))
        connection.commit()
        print(f"Bill generated successfully with invoice number: {invoice_number}")
    except Exception as e:
        print(f"Error generating bill: {e}")
    finally:
        connection.close()

def update_bill_status(bill_id, status, payment_method=None, payment_details=None):
    """Update bill payment status with payment details"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            UPDATE billing 
            SET status = %s, payment_method = %s, payment_details = %s
            WHERE bill_id = %s
            """
            cursor.execute(sql, (status, payment_method, payment_details, bill_id))
        connection.commit()
        print("Bill status updated successfully with payment details!")
    except Exception as e:
        print(f"Error updating bill status: {e}")
    finally:
        connection.close()

def add_medical_record(patient_id, diagnosis, visit_type=None, symptoms=None, 
                       treatment=None, prescription=None, tests_ordered=None,
                       test_results=None, notes=None, follow_up_required=False,
                       follow_up_date=None, date=None, doctor_id=None):
    """Add a medical record for a patient with enhanced details"""
    if date is None:
        date = datetime.now().date()
    
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO medical_record (
                patient_id, doctor_id, visit_type, diagnosis, symptoms,
                treatment, prescription, tests_ordered, test_results,
                notes, follow_up_required, follow_up_date, date
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (
                patient_id, doctor_id, visit_type, diagnosis, symptoms,
                treatment, prescription, tests_ordered, test_results,
                notes, follow_up_required, follow_up_date, date
            ))
        connection.commit()
        print("Comprehensive medical record added successfully!")
    except Exception as e:
        print(f"Error adding medical record: {e}")
    finally:
        connection.close()

def add_user(username, password_hash, salt, role, related_id=None):
    """Add a new user to the system with enhanced security"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO users (
                username, password_hash, salt, role, related_id
            )
            VALUES (%s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (username, password_hash, salt, role, related_id))
        connection.commit()
        print("User added successfully with secure credentials!")
    except Exception as e:
        print(f"Error adding user: {e}")
    finally:
        connection.close()

def authenticate_user(username, password):
    """Authenticate a user with enhanced security"""
    # In a real application, you would use proper password hashing
    # This is a simplified version for demonstration
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT user_id, username, role, related_id
            FROM users
            WHERE username = %s AND password_hash = %s
            """
            cursor.execute(sql, (username, password))
            user = cursor.fetchone()
            
            if user:
                # Update last login time
                update_sql = """
                UPDATE users
                SET last_login = NOW(), login_attempts = 0
                WHERE user_id = %s
                """
                cursor.execute(update_sql, (user['user_id'],))
                connection.commit()
                
                return user
            else:
                # Increment failed login attempts
                update_sql = """
                UPDATE users
                SET login_attempts = login_attempts + 1
                WHERE username = %s
                """
                cursor.execute(update_sql, (username,))
                connection.commit()
                return None
    except Exception as e:
        print(f"Error authenticating user: {e}")
        return None
    finally:
        connection.close()

# View functions for data retrieval and enhanced visualization
def view_patients(export_csv=False, export_excel=False):
    """View all patients with enhanced visualization and export options"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT p.patient_id, p.name, p.age, p.gender, p.blood_type, p.phone, p.email, 
                   p.disease, ip.name AS insurance_provider, p.created_at
            FROM patient p
            LEFT JOIN insurance_provider ip ON p.insurance_provider_id = ip.provider_id
            ORDER BY p.name
            """
            cursor.execute(sql)
            patients = cursor.fetchall()
            
            # Convert to DataFrame
            df = pd.DataFrame(patients)
            
            if df.empty:
                print("No patient records found.")
                return None
            
            # Display in notebook
            display(df.head())
            
            # Export options
            if export_csv:
                csv_path = 'patients_export.csv'
                df.to_csv(csv_path, index=False)
                print(f"Patient data exported to {csv_path}")
            
            if export_excel:
                excel_path = 'patients_export.xlsx'
                df.to_excel(excel_path, index=False)
                print(f"Patient data exported to {excel_path}")
            
            # Create enhanced visualizations
            plt.figure(figsize=(15, 10))
            
            # Age distribution
            plt.subplot(2, 2, 1)
            sns.histplot(df['age'], bins=20, kde=True, color='skyblue')
            plt.title('Age Distribution of Patients')
            plt.xlabel('Age')
            plt.ylabel('Count')
            
            # Gender distribution
            plt.subplot(2, 2, 2)
            gender_counts = df['gender'].value_counts()
            gender_counts.plot(kind='pie', autopct='%1.1f%%', 
                             colors=['lightblue', 'pink', 'lightgray'],
                             startangle=90)
            plt.title('Patient Gender Distribution')
            plt.ylabel('')
            
            # Blood type distribution
            plt.subplot(2, 2, 3)
            if 'blood_type' in df.columns:
                blood_counts = df['blood_type'].value_counts()
                blood_counts.plot(kind='bar', color='lightcoral')
                plt.title('Patient Blood Type Distribution')
                plt.xlabel('Blood Type')
                plt.ylabel('Count')
                plt.xticks(rotation=45)
            else:
                plt.text(0.5, 0.5, 'No blood type data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Disease distribution (top 10)
            plt.subplot(2, 2, 4)
            if 'disease' in df.columns:
                disease_counts = df['disease'].value_counts().head(10)
                disease_counts.plot(kind='barh', color='lightgreen')
                plt.title('Top 10 Patient Conditions')
                plt.xlabel('Count')
                plt.ylabel('Condition')
            else:
                plt.text(0.5, 0.5, 'No disease data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Insurance provider analysis
            if 'insurance_provider' in df.columns and df['insurance_provider'].notna().any():
                plt.figure(figsize=(10, 6))
                insurance_counts = df['insurance_provider'].value_counts().head(10)
                insurance_counts.plot(kind='bar', color='mediumpurple')
                plt.title('Top 10 Insurance Providers')
                plt.xlabel('Provider')
                plt.ylabel('Number of Patients')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            
            return df
    except Exception as e:
        print(f"Error retrieving patients: {e}")
    finally:
        connection.close()

def view_doctors(export_csv=False, export_excel=False):
    """View all doctors with enhanced visualization and export options"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT d.doctor_id, d.name, d.specialization, dept.name AS department, 
                   d.years_of_experience, d.consultation_fee, 
                   d.qualification, d.created_at
            FROM doctor d
            LEFT JOIN department dept ON d.department_id = dept.department_id
            ORDER BY d.name
            """
            cursor.execute(sql)
            doctors = cursor.fetchall()
            
            # Convert to DataFrame
            df = pd.DataFrame(doctors)
            
            if df.empty:
                print("No doctor records found.")
                return None
            
            # Display in notebook
            display(df.head())
            
            # Export options
            if export_csv:
                csv_path = 'doctors_export.csv'
                df.to_csv(csv_path, index=False)
                print(f"Doctor data exported to {csv_path}")
            
            if export_excel:
                excel_path = 'doctors_export.xlsx'
                df.to_excel(excel_path, index=False)
                print(f"Doctor data exported to {excel_path}")
            
            # Create enhanced visualizations
            plt.figure(figsize=(15, 10))
            
            # Specialization distribution
            plt.subplot(2, 2, 1)
            if 'specialization' in df.columns:
                spec_counts = df['specialization'].value_counts().head(10)
                spec_counts.plot(kind='bar', color='lightblue')
                plt.title('Top 10 Doctor Specializations')
                plt.xlabel('Specialization')
                plt.ylabel('Count')
                plt.xticks(rotation=45)
            else:
                plt.text(0.5, 0.5, 'No specialization data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Department distribution
            plt.subplot(2, 2, 2)
            if 'department' in df.columns:
                dept_counts = df['department'].value_counts().head(10)
                dept_counts.plot(kind='pie', autopct='%1.1f%%',
                               colors=sns.color_palette('pastel'),
                               startangle=90)
                plt.title('Doctor Department Distribution')
                plt.ylabel('')
            else:
                plt.text(0.5, 0.5, 'No department data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Experience distribution
            plt.subplot(2, 2, 3)
            if 'years_of_experience' in df.columns:
                sns.boxplot(x=df['years_of_experience'], color='lightgreen')
                plt.title('Distribution of Years of Experience')
                plt.xlabel('Years')
            else:
                plt.text(0.5, 0.5, 'No experience data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Consultation fee distribution
            plt.subplot(2, 2, 4)
            if 'consultation_fee' in df.columns:
                sns.histplot(df['consultation_fee'], bins=20, kde=True, color='salmon')
                plt.title('Consultation Fee Distribution')
                plt.xlabel('Fee (₹)')
                plt.ylabel('Count')
            else:
                plt.text(0.5, 0.5, 'No fee data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Correlation between experience and fee
            if 'years_of_experience' in df.columns and 'consultation_fee' in df.columns:
                plt.figure(figsize=(8, 6))
                sns.regplot(x='years_of_experience', y='consultation_fee', data=df,
                            scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
                plt.title('Experience vs. Consultation Fee')
                plt.xlabel('Years of Experience')
                plt.ylabel('Consultation Fee (₹)')
                plt.tight_layout()
                plt.show()
            
            return df
    except Exception as e:
        print(f"Error retrieving doctors: {e}")
    finally:
        connection.close()

def view_appointments(export_csv=False, export_excel=False, time_frame='month'):
    """View all appointments with enhanced visualization and export options"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT a.appointment_id, p.name as patient_name, d.name as doctor_name, 
                   a.date, a.time, a.duration, a.status, a.reason,
                   d.specialization, p.gender as patient_gender, p.age as patient_age
            FROM appointment a
            JOIN patient p ON a.patient_id = p.patient_id
            JOIN doctor d ON a.doctor_id = d.doctor_id
            ORDER BY a.date DESC, a.time DESC
            """
            cursor.execute(sql)
            appointments = cursor.fetchall()
            
            # Convert to DataFrame
            df = pd.DataFrame(appointments)
            
            if df.empty:
                print("No appointment records found.")
                return None
            
            # Convert date to datetime for better analysis
            df['date'] = pd.to_datetime(df['date'])
            df['time'] = pd.to_datetime(df['time'].astype(str)).dt.time
            df['day_of_week'] = df['date'].dt.day_name()
            df['month'] = df['date'].dt.month_name()
            df['hour'] = pd.to_datetime(df['time'].astype(str)).dt.hour
            
            # Display in notebook
            display(df.head())
            
            # Export options
            if export_csv:
                csv_path = 'appointments_export.csv'
                df.to_csv(csv_path, index=False)
                print(f"Appointment data exported to {csv_path}")
            
            if export_excel:
                excel_path = 'appointments_export.xlsx'
                df.to_excel(excel_path, index=False)
                print(f"Appointment data exported to {excel_path}")
            
            # Create enhanced visualizations
            plt.figure(figsize=(15, 10))
            
            # Appointments by status
            plt.subplot(2, 2, 1)
            status_counts = df['status'].value_counts()
            status_counts.plot(kind='pie', autopct='%1.1f%%', 
                             colors=sns.color_palette('pastel'),
                             startangle=90)
            plt.title('Appointment Status Distribution')
            plt.ylabel('')
            
            # Appointments by day of week
            plt.subplot(2, 2, 2)
            day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            day_counts = df['day_of_week'].value_counts().reindex(day_order)
            day_counts.plot(kind='bar', color='lightblue')
            plt.title('Appointments by Day of Week')
            plt.xlabel('Day')
            plt.ylabel('Number of Appointments')
            
            # Appointments by hour of day
            plt.subplot(2, 2, 3)
            hour_counts = df['hour'].value_counts().sort_index()
            hour_counts.plot(kind='bar', color='lightgreen')
            plt.title('Appointments by Hour of Day')
            plt.xlabel('Hour')
            plt.ylabel('Number of Appointments')
            plt.xticks(range(24), [f"{h}:00" for h in range(24)], rotation=45)
            
            # Appointments by doctor
            plt.subplot(2, 2, 4)
            doctor_counts = df['doctor_name'].value_counts().head(10)
            doctor_counts.plot(kind='barh', color='salmon')
            plt.title('Top 10 Doctors by Appointments')
            plt.xlabel('Number of Appointments')
            plt.ylabel('Doctor')
            
            plt.tight_layout()
            plt.show()
            
            # Time series analysis based on selected time frame
            plt.figure(figsize=(12, 6))
            
            if time_frame == 'day':
                ts_data = df.groupby(df['date'].dt.date).size()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Daily Appointments Trend')
                plt.xlabel('Date')
                plt.ylabel('Number of Appointments')
            elif time_frame == 'week':
                df['week'] = df['date'].dt.strftime('%Y-W%U')
                ts_data = df.groupby('week').size()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Weekly Appointments Trend')
                plt.xlabel('Week')
                plt.ylabel('Number of Appointments')
            elif time_frame == 'month':
                df['month_year'] = df['date'].dt.strftime('%Y-%m')
                ts_data = df.groupby('month_year').size()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Monthly Appointments Trend')
                plt.xlabel('Month')
                plt.ylabel('Number of Appointments')
            else:
                print("Invalid time frame specified. Using 'month' as default.")
                df['month_year'] = df['date'].dt.strftime('%Y-%m')
                ts_data = df.groupby('month_year').size()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Monthly Appointments Trend')
                plt.xlabel('Month')
                plt.ylabel('Number of Appointments')
            
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
            # Duration analysis
            if 'duration' in df.columns:
                plt.figure(figsize=(10, 6))
                sns.boxplot(x='duration', data=df, color='lightblue')
                plt.title('Appointment Duration Distribution')
                plt.xlabel('Duration (minutes)')
                plt.tight_layout()
                plt.show()
            
            return df
    except Exception as e:
        print(f"Error retrieving appointments: {e}")
    finally:
        connection.close()

def view_bills(export_csv=False, export_excel=False, time_frame='month'):
    """View all bills with enhanced visualization and export options"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT b.bill_id, p.name as patient_name, d.name as doctor_name, 
                   b.invoice_number, b.amount, b.tax, b.discount, b.total_amount,
                   b.date, b.due_date, b.status, b.payment_method,
                   ip.name AS insurance_provider, d.specialization
            FROM billing b
            JOIN patient p ON b.patient_id = p.patient_id
            JOIN doctor d ON b.doctor_id = d.doctor_id
            LEFT JOIN insurance_provider ip ON p.insurance_provider_id = ip.provider_id
            ORDER BY b.date DESC
            """
            cursor.execute(sql)
            bills = cursor.fetchall()
            
            # Convert to DataFrame
            df = pd.DataFrame(bills)
            
            if df.empty:
                print("No billing records found.")
                return None
            
            # Convert date to datetime for better analysis
            df['date'] = pd.to_datetime(df['date'])
            df['due_date'] = pd.to_datetime(df['due_date'])
            df['month'] = df['date'].dt.month_name()
            df['year'] = df['date'].dt.year
            
            # Calculate days to payment
            df['days_to_pay'] = (df['due_date'] - df['date']).dt.days
            
            # Display in notebook
            display(df.head())
            
            # Export options
            if export_csv:
                csv_path = 'bills_export.csv'
                df.to_csv(csv_path, index=False)
                print(f"Bill data exported to {csv_path}")
            
            if export_excel:
                excel_path = 'bills_export.xlsx'
                df.to_excel(excel_path, index=False)
                print(f"Bill data exported to {excel_path}")
            
            # Create enhanced visualizations
            plt.figure(figsize=(15, 10))
            
            # Bill status distribution
            plt.subplot(2, 2, 1)
            status_counts = df['status'].value_counts()
            status_counts.plot(kind='pie', autopct='%1.1f%%', 
                             colors=sns.color_palette('pastel'),
                             startangle=90)
            plt.title('Bill Status Distribution')
            plt.ylabel('')
            
            # Payment method distribution
            plt.subplot(2, 2, 2)
            if 'payment_method' in df.columns:
                method_counts = df['payment_method'].value_counts()
                method_counts.plot(kind='bar', color='lightblue')
                plt.title('Payment Method Distribution')
                plt.xlabel('Payment Method')
                plt.ylabel('Count')
                plt.xticks(rotation=45)
            else:
                plt.text(0.5, 0.5, 'No payment method data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Amount distribution
            plt.subplot(2, 2, 3)
            sns.histplot(df['total_amount'], bins=20, kde=True, color='lightgreen')
            plt.title('Total Amount Distribution')
            plt.xlabel('Amount (₹)')
            plt.ylabel('Count')
            
            # Days to pay distribution
            plt.subplot(2, 2, 4)
            sns.boxplot(x=df['days_to_pay'], color='salmon')
            plt.title('Payment Terms Distribution')
            plt.xlabel('Days to Pay')
            
            plt.tight_layout()
            plt.show()
            
            # Time series analysis based on selected time frame
            plt.figure(figsize=(12, 6))
            
            if time_frame == 'day':
                ts_data = df.groupby(df['date'].dt.date)['total_amount'].sum()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Daily Revenue Trend')
                plt.xlabel('Date')
                plt.ylabel('Revenue (₹)')
            elif time_frame == 'week':
                df['week'] = df['date'].dt.strftime('%Y-W%U')
                ts_data = df.groupby('week')['total_amount'].sum()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Weekly Revenue Trend')
                plt.xlabel('Week')
                plt.ylabel('Revenue (₹)')
            elif time_frame == 'month':
                df['month_year'] = df['date'].dt.strftime('%Y-%m')
                ts_data = df.groupby('month_year')['total_amount'].sum()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Monthly Revenue Trend')
                plt.xlabel('Month')
                plt.ylabel('Revenue (₹)')
            else:
                print("Invalid time frame specified. Using 'month' as default.")
                df['month_year'] = df['date'].dt.strftime('%Y-%m')
                ts_data = df.groupby('month_year').size()
                ts_data.plot(kind='line', marker='o', color='mediumpurple')
                plt.title('Monthly Appointments Trend')
                plt.xlabel('Month')
                plt.ylabel('Number of Appointments')
            
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
            # Revenue by doctor
            if not df.empty:
                plt.figure(figsize=(12, 6))
                revenue_by_doctor = df.groupby('doctor_name')['total_amount'].sum().sort_values(ascending=False).head(10)
                revenue_by_doctor.plot(kind='bar', color='lightblue')
                plt.title('Top 10 Doctors by Revenue Generated')
                plt.xlabel('Doctor')
                plt.ylabel('Total Revenue (₹)')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            
            # Insurance provider analysis
            if 'insurance_provider' in df.columns and df['insurance_provider'].notna().any():
                plt.figure(figsize=(12, 6))
                insurance_revenue = df.groupby('insurance_provider')['total_amount'].sum().sort_values(ascending=False).head(10)
                insurance_revenue.plot(kind='bar', color='lightgreen')
                plt.title('Top 10 Insurance Providers by Revenue')
                plt.xlabel('Insurance Provider')
                plt.ylabel('Total Revenue (₹)')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            
            return df
    except Exception as e:
        print(f"Error retrieving bills: {e}")
    finally:
        connection.close()

def view_medical_records(export_csv=False, export_excel=False):
    """View all medical records with enhanced visualization and export options"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT mr.record_id, p.name as patient_name, d.name as doctor_name,
                   mr.visit_type, mr.diagnosis, mr.date, mr.follow_up_required,
                   mr.follow_up_date, p.age as patient_age, p.gender as patient_gender,
                   d.specialization
            FROM medical_record mr
            JOIN patient p ON mr.patient_id = p.patient_id
            LEFT JOIN doctor d ON mr.doctor_id = d.doctor_id
            ORDER BY mr.date DESC
            """
            cursor.execute(sql)
            records = cursor.fetchall()
            
            # Convert to DataFrame
            df = pd.DataFrame(records)
            
            if df.empty:
                print("No medical records found.")
                return None
            
            # Convert date to datetime for better analysis
            df['date'] = pd.to_datetime(df['date'])
            df['month'] = df['date'].dt.month_name()
            df['year'] = df['date'].dt.year
            
            # Display in notebook
            display(df.head())
            
            # Export options
            if export_csv:
                csv_path = 'medical_records_export.csv'
                df.to_csv(csv_path, index=False)
                print(f"Medical record data exported to {csv_path}")
            
            if export_excel:
                excel_path = 'medical_records_export.xlsx'
                df.to_excel(excel_path, index=False)
                print(f"Medical record data exported to {excel_path}")
            
            # Create enhanced visualizations
            plt.figure(figsize=(15, 10))
            
            # Visit type distribution
            plt.subplot(2, 2, 1)
            if 'visit_type' in df.columns:
                visit_counts = df['visit_type'].value_counts()
                visit_counts.plot(kind='pie', autopct='%1.1f%%', 
                                 colors=sns.color_palette('pastel'),
                                 startangle=90)
                plt.title('Visit Type Distribution')
                plt.ylabel('')
            else:
                plt.text(0.5, 0.5, 'No visit type data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Follow-up required
            plt.subplot(2, 2, 2)
            if 'follow_up_required' in df.columns:
                follow_up_counts = df['follow_up_required'].value_counts()
                follow_up_counts.plot(kind='bar', color=['lightgreen', 'lightcoral'])
                plt.title('Follow-up Required')
                plt.xlabel('Follow-up Required')
                plt.ylabel('Count')
                plt.xticks([0, 1], ['No', 'Yes'], rotation=0)
            else:
                plt.text(0.5, 0.5, 'No follow-up data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Records by month
            plt.subplot(2, 2, 3)
            if not df.empty:
                monthly_counts = df.groupby(['year', 'month']).size().unstack().T
                monthly_counts = monthly_counts.reindex(calendar.month_name[1:], axis=0)
                monthly_counts.plot(kind='bar', stacked=True, 
                                     colormap='viridis', 
                                     ax=plt.gca())
                plt.title('Medical Records by Month')
                plt.xlabel('Month')
                plt.ylabel('Number of Records')
                plt.legend(title='Year')
            else:
                plt.text(0.5, 0.5, 'No date data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Top diagnoses
            plt.subplot(2, 2, 4)
            if 'diagnosis' in df.columns:
                # Extract the first diagnosis if multiple are listed
                df['primary_diagnosis'] = df['diagnosis'].str.split(',').str[0].str.strip()
                diagnosis_counts = df['primary_diagnosis'].value_counts().head(10)
                diagnosis_counts.plot(kind='barh', color='lightblue')
                plt.title('Top 10 Diagnoses')
                plt.xlabel('Count')
                plt.ylabel('Diagnosis')
            else:
                plt.text(0.5, 0.5, 'No diagnosis data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Age distribution by diagnosis
            if 'primary_diagnosis' in df.columns and 'patient_age' in df.columns:
                plt.figure(figsize=(12, 6))
                top_diagnoses = df['primary_diagnosis'].value_counts().head(5).index
                filtered_df = df[df['primary_diagnosis'].isin(top_diagnoses)]
                sns.boxplot(x='primary_diagnosis', y='patient_age', data=filtered_df,
                            palette='pastel')
                plt.title('Age Distribution by Top Diagnoses')
                plt.xlabel('Diagnosis')
                plt.ylabel('Age')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            
            return df
    except Exception as e:
        print(f"Error retrieving medical records: {e}")
    finally:
        connection.close()

# Enhanced reporting functions

def generate_financial_report(time_frame='month'):
    """Generate a comprehensive financial report"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # Get billing data
            sql = """
            SELECT date, total_amount, status, payment_method
            FROM billing
            ORDER BY date
            """
            cursor.execute(sql)
            bills = cursor.fetchall()
            
            df = pd.DataFrame(bills)
            
            if df.empty:
                print("No billing data available for financial report.")
                return None
            
            df['date'] = pd.to_datetime(df['date'])
            
            # Set up time-based grouping
            if time_frame == 'day':
                df['time_group'] = df['date'].dt.date
                title_suffix = 'Daily'
            elif time_frame == 'week':
                df['time_group'] = df['date'].dt.strftime('%Y-W%U')
                title_suffix = 'Weekly'
            elif time_frame == 'month':
                df['time_group'] = df['date'].dt.strftime('%Y-%m')
                title_suffix = 'Monthly'
            else:
                print("Invalid time frame specified. Using 'month' as default.")
                df['time_group'] = df['date'].dt.strftime('%Y-%m')
                title_suffix = 'Monthly'
            
            # Calculate metrics
            report = df.groupby('time_group').agg({
                'total_amount': ['sum', 'count', 'mean', 'median', 'max', 'min']
            })
            
            report.columns = ['Total Revenue', 'Number of Bills', 'Average Bill', 
                              'Median Bill', 'Largest Bill', 'Smallest Bill']
            
            # Payment status analysis
            status_report = pd.crosstab(df['time_group'], df['status'], 
                                        values=df['total_amount'], 
                                        aggfunc='sum', normalize='index') * 100
            
            # Payment method analysis
            method_report = pd.crosstab(df['time_group'], df['payment_method'], 
                                        values=df['total_amount'], 
                                        aggfunc='sum', normalize='index') * 100
            
            # Display reports
            print(f"\n{title_suffix} Financial Summary:")
            display(report)
            
            print(f"\n{title_suffix} Payment Status Analysis (% of revenue):")
            display(status_report)
            
            print(f"\n{title_suffix} Payment Method Analysis (% of revenue):")
            display(method_report)
            
            # Visualization
            plt.figure(figsize=(15, 10))
            
            # Revenue trend
            plt.subplot(2, 2, 1)
            report['Total Revenue'].plot(kind='line', marker='o', color='green')
            plt.title(f'{title_suffix} Revenue Trend')
            plt.ylabel('Revenue (₹)')
            plt.grid(True)
            
            # Number of bills
            plt.subplot(2, 2, 2)
            report['Number of Bills'].plot(kind='bar', color='blue')
            plt.title(f'{title_suffix} Number of Bills')
            plt.ylabel('Count')
            
            # Average bill amount
            plt.subplot(2, 2, 3)
            report['Average Bill'].plot(kind='line', marker='o', color='orange')
            plt.title(f'{title_suffix} Average Bill Amount')
            plt.ylabel('Amount (₹)')
            plt.grid(True)
            
            # Payment status distribution
            plt.subplot(2, 2, 4)
            status_report.plot(kind='area', stacked=True, ax=plt.gca(),
                               colormap='Pastel1')
            plt.title(f'{title_suffix} Payment Status Distribution')
            plt.ylabel('Percentage (%)')
            plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
            
            plt.tight_layout()
            plt.show()
            
            return report, status_report, method_report
    except Exception as e:
        print(f"Error generating financial report: {e}")
    finally:
        connection.close()

def generate_operational_report(time_frame='month'):
    """Generate a comprehensive operational report"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # Get appointment data
            sql = """
            SELECT a.date, a.status, d.specialization, p.gender, p.age, dept.name AS department
            FROM appointment a
            JOIN doctor d ON a.doctor_id = d.doctor_id
            JOIN patient p ON a.patient_id = p.patient_id
            LEFT JOIN department dept ON d.department_id = dept.department_id
            ORDER BY a.date
            """
            cursor.execute(sql)
            appointments = cursor.fetchall()
            
            df = pd.DataFrame(appointments)
            
            if df.empty:
                print("No appointment data available for operational report.")
                return None
            
            df['date'] = pd.to_datetime(df['date'])
            
            # Set up time-based grouping
            if time_frame == 'day':
                df['time_group'] = df['date'].dt.date
                title_suffix = 'Daily'
            elif time_frame == 'week':
                df['time_group'] = df['date'].dt.strftime('%Y-W%U')
                title_suffix = 'Weekly'
            elif time_frame == 'month':
                df['time_group'] = df['date'].dt.strftime('%Y-%m')
                title_suffix = 'Monthly'
            else:
                print("Invalid time frame specified. Using 'month' as default.")
                df['time_group'] = df['date'].dt.strftime('%Y-%m')
                title_suffix = 'Monthly'
            
            # Calculate metrics
            report = df.groupby('time_group').agg({
                'date': 'count',
                'age': 'mean'
            }).rename(columns={'date': 'Number of Appointments', 'age': 'Average Patient Age'})
            
            # Appointment status analysis
            status_report = pd.crosstab(df['time_group'], df['status'], 
                                        normalize='index') * 100
            
            # Specialization analysis
            spec_report = pd.crosstab(df['time_group'], df['specialization']).T
            
            # Display reports
            print(f"\n{title_suffix} Operational Summary:")
            display(report)
            
            print(f"\n{title_suffix} Appointment Status Analysis (%):")
            display(status_report)
            
            print(f"\n{title_suffix} Specialization Distribution:")
            display(spec_report)
            
            # Visualization
            plt.figure(figsize=(15, 10))
            
            # Appointment trend
            plt.subplot(2, 2, 1)
            report['Number of Appointments'].plot(kind='line', marker='o', color='blue')
            plt.title(f'{title_suffix} Appointment Trend')
            plt.ylabel('Number of Appointments')
            plt.grid(True)
            
            # Average patient age
            plt.subplot(2, 2, 2)
            report['Average Patient Age'].plot(kind='line', marker='o', color='green')
            plt.title(f'{title_suffix} Average Patient Age')
            plt.ylabel('Age')
            plt.grid(True)
            
            # Appointment status distribution
            plt.subplot(2, 2, 3)
            status_report.plot(kind='area', stacked=True, ax=plt.gca(),
                               colormap='Pastel2')
            plt.title(f'{title_suffix} Appointment Status Distribution')
            plt.ylabel('Percentage (%)')
            plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
            
            # Top specializations
            plt.subplot(2, 2, 4)
            spec_report.sum(axis=1).sort_values(ascending=False).head(10).plot(
                kind='barh', color='purple')
            plt.title('Top 10 Specializations')
            plt.xlabel('Total Appointments')
            
            plt.tight_layout()
            plt.show()
            
            return report, status_report, spec_report
    except Exception as e:
        print(f"Error generating operational report: {e}")
    finally:
        connection.close()

def generate_doctor_performance_report():
    """Generate a doctor performance report with multiple metrics"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # Get doctor performance data
            sql = """
            SELECT 
                d.doctor_id,
                d.name AS doctor_name,
                d.specialization,
                d.years_of_experience,
                d.consultation_fee,
                COUNT(DISTINCT a.appointment_id) AS total_appointments,
                COUNT(DISTINCT CASE WHEN a.status = 'Completed' THEN a.appointment_id END) AS completed_appointments,
                COUNT(DISTINCT CASE WHEN a.status = 'Cancelled' THEN a.appointment_id END) AS cancelled_appointments,
                COUNT(DISTINCT CASE WHEN a.status = 'No-Show' THEN a.appointment_id END) AS no_show_appointments,
                COUNT(DISTINCT mr.record_id) AS medical_records,
                COUNT(DISTINCT b.bill_id) AS bills_generated,
                IFNULL(SUM(b.total_amount), 0) AS total_revenue
            FROM doctor d
            LEFT JOIN appointment a ON d.doctor_id = a.doctor_id
            LEFT JOIN medical_record mr ON d.doctor_id = mr.doctor_id
            LEFT JOIN billing b ON d.doctor_id = b.doctor_id
            GROUP BY d.doctor_id, d.name, d.specialization, d.years_of_experience, d.consultation_fee
            ORDER BY total_revenue DESC
            """
            cursor.execute(sql)
            doctors = cursor.fetchall()
            
            df = pd.DataFrame(doctors)
            
            if df.empty:
                print("No doctor data available for performance report.")
                return None
            
            # Calculate performance metrics
            df['completion_rate'] = (df['completed_appointments'] / df['total_appointments']) * 100
            df['cancellation_rate'] = (df['cancelled_appointments'] / df['total_appointments']) * 100
            df['no_show_rate'] = (df['no_show_appointments'] / df['total_appointments']) * 100
            df['revenue_per_appointment'] = df['total_revenue'] / df['total_appointments']
            
            # Display report
            print("\nDoctor Performance Report:")
            display(df)
            
            # Visualization
            plt.figure(figsize=(15, 10))
            
            # Revenue by doctor
            plt.subplot(2, 2, 1)
            df.sort_values('total_revenue', ascending=False).head(10).plot(
                x='doctor_name', y='total_revenue', kind='bar', color='green')
            plt.title('Top 10 Doctors by Revenue')
            plt.ylabel('Total Revenue (₹)')
            plt.xticks(rotation=45)
            
            # Completion rate vs. experience
            plt.subplot(2, 2, 2)
            sns.regplot(x='years_of_experience', y='completion_rate', data=df,
                        scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
            plt.title('Experience vs. Appointment Completion Rate')
            plt.xlabel('Years of Experience')
            plt.ylabel('Completion Rate (%)')
            
            # Specialization analysis
            plt.subplot(2, 2, 3)
            spec_stats = df.groupby('specialization').agg({
                'total_revenue': 'sum',
                'total_appointments': 'sum'
            }).sort_values('total_revenue', ascending=False).head(10)
            
            spec_stats['total_revenue'].plot(kind='bar', color='blue')
            plt.title('Top 10 Specializations by Revenue')
            plt.ylabel('Total Revenue (₹)')
            plt.xticks(rotation=45)
            
            # Performance matrix
            plt.subplot(2, 2, 4)
            sns.scatterplot(x='total_appointments', y='revenue_per_appointment',
                            size='years_of_experience', hue='specialization',
                            data=df, sizes=(20, 200), alpha=0.6)
            plt.title('Doctor Performance Matrix')
            plt.xlabel('Total Appointments')
            plt.ylabel('Revenue per Appointment (₹)')
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            
            plt.tight_layout()
            plt.show()
            
            return df
    except Exception as e:
        print(f"Error generating doctor performance report: {e}")
    finally:
        connection.close()

def generate_patient_statistics_report():
    """Generate a comprehensive patient statistics report"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # Get patient data
            sql = """
            SELECT 
                p.patient_id,
                p.name as patient_name,
                p.age,
                p.gender,
                p.blood_type,
                p.disease,
                ip.name AS insurance_provider,
                COUNT(DISTINCT a.appointment_id) AS total_appointments,
                COUNT(DISTINCT mr.record_id) AS medical_records,
                COUNT(DISTINCT b.bill_id) AS total_bills,
                IFNULL(SUM(b.total_amount), 0) AS total_amount_billed,
                IFNULL(SUM(CASE WHEN b.status = 'Paid' THEN b.total_amount ELSE 0 END), 0) AS total_amount_paid
            FROM patient p
            LEFT JOIN appointment a ON p.patient_id = a.patient_id
            LEFT JOIN medical_record mr ON p.patient_id = mr.patient_id
            LEFT JOIN billing b ON p.patient_id = b.patient_id
            LEFT JOIN insurance_provider ip ON p.insurance_provider_id = ip.provider_id
            GROUP BY p.patient_id, p.name, p.age, p.gender, p.blood_type, p.disease, p.insurance_provider_id
            ORDER BY total_amount_billed DESC
            """
            cursor.execute(sql)
            patients = cursor.fetchall()
            
            df = pd.DataFrame(patients)
            
            if df.empty:
                print("No patient data available for statistics report.")
                return None
            
            # Calculate additional metrics
            df['outstanding_balance'] = df['total_amount_billed'] - df['total_amount_paid']
            
            # Display report
            print("\nPatient Statistics Report:")
            display(df.head(20))  # Show top 20 patients by amount billed
            
            # Visualization
            plt.figure(figsize=(15, 10))
            
            # Age distribution
            plt.subplot(2, 2, 1)
            sns.histplot(df['age'], bins=20, kde=True, color='skyblue')
            plt.title('Patient Age Distribution')
            plt.xlabel('Age')
            plt.ylabel('Count')
            
            # Gender distribution
            plt.subplot(2, 2, 2)
            gender_counts = df['gender'].value_counts()
            gender_counts.plot(kind='pie', autopct='%1.1f%%', 
                             colors=['lightblue', 'pink', 'lightgray'],
                             startangle=90)
            plt.title('Patient Gender Distribution')
            plt.ylabel('')
            
            # Top diseases
            plt.subplot(2, 2, 3)
            if 'disease' in df.columns:
                disease_counts = df['disease'].value_counts().head(10)
                disease_counts.plot(kind='barh', color='lightgreen')
                plt.title('Top 10 Patient Conditions')
                plt.xlabel('Count')
                plt.ylabel('Condition')
            else:
                plt.text(0.5, 0.5, 'No disease data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            # Outstanding balance analysis
            plt.subplot(2, 2, 4)
            sns.boxplot(x=df['outstanding_balance'], color='salmon')
            plt.title('Distribution of Outstanding Balances')
            plt.xlabel('Outstanding Balance (₹)')
            
            plt.tight_layout()
            plt.show()
            
            # Insurance provider analysis
            if 'insurance_provider' in df.columns and df['insurance_provider'].notna().any():
                plt.figure(figsize=(12, 6))
                insurance_stats = df.groupby('insurance_provider').agg({
                    'patient_id': 'count',
                    'total_amount_billed': 'sum'
                }).sort_values('total_amount_billed', ascending=False).head(10)
                
                insurance_stats['total_amount_billed'].plot(kind='bar', color='purple')
                plt.title('Top 10 Insurance Providers by Total Billed Amount')
                plt.ylabel('Total Amount Billed (₹)')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            
            return df
    except Exception as e:
        print(f"Error generating patient statistics report: {e}")
    finally:
        connection.close()

# Helper functions for data analysis

def analyze_patient_demographics():
    """Generate a detailed analysis of patient demographics"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT age, gender, blood_type, disease, ip.name AS insurance_provider
            FROM patient p
            LEFT JOIN insurance_provider ip ON p.insurance_provider_id = ip.provider_id
            """
            cursor.execute(sql)
            patients = cursor.fetchall()
            
            df = pd.DataFrame(patients)
            
            if df.empty:
                print("No patient data available for demographic analysis.")
                return None
            
            # Age analysis
            age_stats = df['age'].describe()
            
            # Gender distribution
            gender_dist = df['gender'].value_counts(normalize=True) * 100
            
            # Blood type distribution
            blood_dist = df['blood_type'].value_counts(normalize=True) * 100
            
            # Disease distribution
            disease_dist = df['disease'].value_counts().head(10)
            
            # Insurance distribution
            insurance_dist = df['insurance_provider'].value_counts().head(10)
            
            # Display statistics
            print("\nPatient Age Statistics:")
            display(age_stats)
            
            print("\nGender Distribution (%):")
            display(gender_dist)
            
            print("\nBlood Type Distribution (%):")
            display(blood_dist)
            
            print("\nTop 10 Conditions:")
            display(disease_dist)
            
            print("\nTop 10 Insurance Providers:")
            display(insurance_dist)
            
            # Visualization
            plt.figure(figsize=(15, 12))
            
            # Age distribution with disease highlight
            plt.subplot(3, 2, 1)
            sns.boxplot(x='age', data=df, color='lightblue')
            plt.title('Age Distribution')
            plt.xlabel('Age')
            
            plt.subplot(3, 2, 2)
            gender_dist.plot(kind='pie', autopct='%1.1f%%', 
                             colors=['lightblue', 'pink', 'lightgray'],
                             startangle=90)
            plt.title('Gender Distribution')
            plt.ylabel('')
            
            plt.subplot(3, 2, 3)
            if not df['blood_type'].isna().all():
                blood_dist.plot(kind='bar', color='lightcoral')
                plt.title('Blood Type Distribution')
                plt.ylabel('Percentage')
                plt.xticks(rotation=45)
            else:
                plt.text(0.5, 0.5, 'No blood type data available', 
                         ha='center', va='center')
                plt.axis('off')
            
            plt.subplot(3, 2, 4)
            disease_dist.plot(kind='barh', color='lightgreen')
            plt.title('Top 10 Conditions')
            plt.xlabel('Count')
            
            plt.subplot(3, 2, 5)
            insurance_dist.plot(kind='bar', color='mediumpurple')
            plt.title('Top 10 Insurance Providers')
            plt.xlabel('Provider')
            plt.ylabel('Count')
            plt.xticks(rotation=45)
            
            plt.tight_layout()
            plt.show()
            
            # Age distribution by gender
            plt.figure(figsize=(10, 6))
            sns.boxplot(x='gender', y='age', data=df,
                        palette=['lightblue', 'pink', 'lightgray'])
            plt.title('Age Distribution by Gender')
            plt.xlabel('Gender')
            plt.ylabel('Age')
            plt.tight_layout()
            plt.show()
            
            return {
                'age_stats': age_stats,
                'gender_dist': gender_dist,
                'blood_dist': blood_dist,
                'disease_dist': disease_dist,
                'insurance_dist': insurance_dist
            }
    except Exception as e:
        print(f"Error analyzing patient demographics: {e}")
    finally:
        connection.close()

def analyze_appointment_patterns():
    """Analyze patterns in appointment scheduling"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT
                a.date,
                a.time,
                a.status,
                d.specialization,
                p.gender as patient_gender,
                p.age as patient_age
            FROM appointment a
            JOIN doctor d ON a.doctor_id = d.doctor_id
            JOIN patient p ON a.patient_id = p.patient_id
            """
            cursor.execute(sql)
            appointments = cursor.fetchall()
            
            df = pd.DataFrame(appointments)
            
            if df.empty:
                print("No appointment data available for pattern analysis.")
                return None
            
            # Convert date/time fields
            df['date'] = pd.to_datetime(df['date'])
            df['time'] = pd.to_datetime(df['time'].astype(str)).dt.time
            df['day_of_week'] = df['date'].dt.day_name()
            df['hour'] = pd.to_datetime(df['time'].astype(str)).dt.hour
            df['month'] = df['date'].dt.month_name()
            
            # Day of week analysis
            day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            day_counts = df['day_of_week'].value_counts().reindex(day_order)
            
            # Hour of day analysis
            hour_counts = df['hour'].value_counts().sort_index()
            
            # Status analysis
            status_counts = df['status'].value_counts()
            
            # Specialization analysis
            spec_counts = df['specialization'].value_counts().head(10)
            
            # Age/gender analysis
            gender_age = df.groupby('patient_gender')['patient_age'].mean()
            
            # Display statistics
            print("\nAppointments by Day of Week:")
            display(day_counts)
            
            print("\nAppointments by Hour of Day:")
            display(hour_counts)
            
            print("\nAppointment Status Distribution:")
            display(status_counts)
            
            print("\nTop 10 Specializations by Appointments:")
            display(spec_counts)
            
            print("\nAverage Patient Age by Gender:")
            display(gender_age)
            
            # Visualization
            plt.figure(figsize=(15, 12))
            
            # Day of week pattern
            plt.subplot(3, 2, 1)
            day_counts.plot(kind='bar', color='lightblue')
            plt.title('Appointments by Day of Week')
            plt.xlabel('Day')
            plt.ylabel('Number of Appointments')
            
            # Hour of day pattern
            plt.subplot(3, 2, 2)
            hour_counts.plot(kind='bar', color='lightgreen')
            plt.title('Appointments by Hour of Day')
            plt.xlabel('Hour')
            plt.ylabel('Number of Appointments')
            plt.xticks(range(24), [f"{h}:00" for h in range(24)], rotation=45)
            
            # Status distribution
            plt.subplot(3, 2, 3)
            status_counts.plot(kind='pie', autopct='%1.1f%%',
                             colors=sns.color_palette('pastel'),
                             startangle=90)
            plt.title('Appointment Status Distribution')
            plt.ylabel('')
            
            # Specialization distribution
            plt.subplot(3, 2, 4)
            spec_counts.plot(kind='barh', color='salmon')
            plt.title('Top 10 Specializations')
            plt.xlabel('Number of Appointments')
            
            # Age by gender
            plt.subplot(3, 2, 5)
            gender_age.plot(kind='bar', color=['lightblue', 'pink', 'lightgray'])
            plt.title('Average Patient Age by Gender')
            plt.xlabel('Gender')
            plt.ylabel('Average Age')
            
            # Time series of appointments
            plt.subplot(3, 2, 6)
            monthly_counts = df.groupby(df['date'].dt.to_period('M')).size()
            monthly_counts.plot(kind='line', marker='o', color='purple')
            plt.title('Monthly Appointment Trend')
            plt.xlabel('Month')
            plt.ylabel('Number of Appointments')
            
            plt.tight_layout()
            plt.show()
            
            return {
                'day_counts': day_counts,
                'hour_counts': hour_counts,
                'status_counts': status_counts,
                'spec_counts': spec_counts,
                'gender_age': gender_age
            }
    except Exception as e:
        print(f"Error analyzing appointment patterns: {e}")
    finally:
        connection.close()

def analyze_revenue_streams():
    """Analyze revenue streams and patterns"""
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            SELECT 
                b.date,
                b.total_amount,
                b.status,
                b.payment_method,
                d.specialization,
                ip.name AS insurance_provider
            FROM billing b
            JOIN doctor d ON b.doctor_id = d.doctor_id
            JOIN patient p ON b.patient_id = p.patient_id
            LEFT JOIN insurance_provider ip ON p.insurance_provider_id = ip.provider_id
            """
            cursor.execute(sql)
            bills = cursor.fetchall()
            
            df = pd.DataFrame(bills)
            
            if df.empty:
                print("No billing data available for revenue analysis.")
                return None
            
            # Convert date field
            df['date'] = pd.to_datetime(df['date'])
            df['month'] = df['date'].dt.month_name()
            df['day_of_week'] = df['date'].dt.day_name()
            
            # Overall revenue stats
            revenue_stats = df['total_amount'].describe()
            
            # Revenue by month
            monthly_revenue = df.groupby(df['date'].dt.to_period('M'))['total_amount'].sum()
            
            # Revenue by payment method
            payment_revenue = df.groupby('payment_method')['total_amount'].sum().sort_values(ascending=False)
            
            # Revenue by specialization
            spec_revenue = df.groupby('specialization')['total_amount'].sum().sort_values(ascending=False).head(10)
            
            # Revenue by insurance provider
            insurance_revenue = df.groupby('insurance_provider')['total_amount'].sum().sort_values(ascending=False).head(10)
            
            # Display statistics
            print("\nRevenue Statistics:")
            display(revenue_stats)
            
            print("\nMonthly Revenue:")
            display(monthly_revenue)
            
            print("\nRevenue by Payment Method:")
            display(payment_revenue)
            
            print("\nRevenue by Specialization (Top 10):")
            display(spec_revenue)
            
            print("\nRevenue by Insurance Provider (Top 10):")
            display(insurance_revenue)
            
            # Visualization
            plt.figure(figsize=(15, 12))
            
            # Revenue distribution
            plt.subplot(3, 2, 1)
            sns.histplot(df['total_amount'], bins=20, kde=True, color='green')
            plt.title('Revenue Distribution per Bill')
            plt.xlabel('Amount (₹)')
            plt.ylabel('Count')
            
            # Monthly revenue trend
            plt.subplot(3, 2, 2)
            monthly_revenue.plot(kind='line', marker='o', color='blue')
            plt.title('Monthly Revenue Trend')
            plt.xlabel('Month')
            plt.ylabel('Revenue (₹)')
            
            # Payment method contribution
            plt.subplot(3, 2, 3)
            payment_revenue.plot(kind='pie', autopct='%1.1f%%',
                               colors=sns.color_palette('pastel'),
                               startangle=90)
            plt.title('Revenue by Payment Method')
            plt.ylabel('')
            
            # Specialization revenue
            plt.subplot(3, 2, 4)
            spec_revenue.plot(kind='barh', color='purple')
            plt.title('Top 10 Specializations by Revenue')
            plt.xlabel('Revenue (₹)')
            
            # Insurance provider revenue
            plt.subplot(3, 2, 5)
            insurance_revenue.plot(kind='bar', color='orange')
            plt.title('Top 10 Insurance Providers by Revenue')
            plt.xlabel('Provider')
            plt.ylabel('Revenue (₹)')
            plt.xticks(rotation=45)
            
            # Day of week revenue
            plt.subplot(3, 2, 6)
            day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            day_revenue = df.groupby('day_of_week')['total_amount'].sum().reindex(day_order)
            day_revenue.plot(kind='bar', color='teal')
            plt.title('Revenue by Day of Week')
            plt.xlabel('Day')
            plt.ylabel('Revenue (₹)')
            
            plt.tight_layout()
            plt.show()
            
            return {
                'revenue_stats': revenue_stats,
                'monthly_revenue': monthly_revenue,
                'payment_revenue': payment_revenue,
                'spec_revenue': spec_revenue,
                'insurance_revenue': insurance_revenue
            }
    except Exception as e:
        print(f"Error analyzing revenue streams: {e}")
    finally:
        connection.close()

# Menu functions (moved outside of main_menu)

def patient_management_menu():
    """Display patient management menu and handle user input"""
    while True:
        print("\n=== PATIENT MANAGEMENT ===")
        print("1. Add New Patient")
        print("2. Update Patient Information")
        print("3. View All Patients")
        print("4. Search Patient")
        print("5. Delete Patient")
        print("6. Back to Main Menu")
        
        choice = input("Enter your choice (1-6): ")
        
        if choice == '1':
            name = input("Enter patient name: ")
            age = int(input("Enter patient age: "))
            gender = input("Enter patient gender (Male/Female/Other): ")
            blood_type = input("Enter blood type (A+, A-, B+, B-, AB+, AB-, O+, O-, or leave blank): ")
            address = input("Enter patient address (optional): ")
            phone = input("Enter patient phone (optional): ")
            email = input("Enter patient email (optional): ")
            insurance_provider_id = input("Enter insurance provider ID (optional): ")
            insurance_provider_id = int(insurance_provider_id) if insurance_provider_id.isdigit() else None
            insurance_policy_number = input("Enter insurance policy number (optional): ")
            primary_physician = input("Enter primary physician (optional): ")
            emergency_contact = input("Enter emergency contact name (optional): ")
            emergency_phone = input("Enter emergency phone (optional): ")
            medical_history = input("Enter medical history (optional): ")
            current_medications = input("Enter current medications (optional): ")
            allergies = input("Enter allergies (optional): ")
            disease = input("Enter disease/condition (optional): ")
            
            add_patient(
                name=name, age=age, gender=gender, blood_type=blood_type if blood_type else None,
                address=address if address else None, phone=phone if phone else None,
                email=email if email else None, insurance_provider_id=insurance_provider_id,
                insurance_policy_number=insurance_policy_number if insurance_policy_number else None, primary_physician=primary_physician if primary_physician else None,
                emergency_contact=emergency_contact if emergency_contact else None, emergency_phone=emergency_phone if emergency_phone else None,
                medical_history=medical_history if medical_history else None, current_medications=current_medications if current_medications else None,
                allergies=allergies if allergies else None, disease=disease if disease else None
            )
            
        elif choice == '2':
            try:
                patient_id = int(input("Enter patient ID to update: "))
            except ValueError:
                print("Invalid input. Patient ID must be a number.")
                continue

            connection = get_db_connection()
            try:
                with connection.cursor() as cursor:
                    sql = "SELECT * FROM patient WHERE patient_id = %s"
                    cursor.execute(sql, (patient_id,))
                    patient = cursor.fetchone()

                    if not patient:
                        print("Patient not found!")
                        continue

                    print("\nCurrent Patient Information:")
                    for key, value in patient.items():
                        print(f"{key}: {value}")

                    print("\nEnter new information (leave blank to keep current value):")
                    name = input(f"Name [{patient.get('name')}]: ").strip() or patient.get('name')
                    age_input = input(f"Age [{patient.get('age')}]: ").strip()
                    try:
                        age = int(age_input) if age_input else patient.get('age')
                    except ValueError:
                        print("Invalid age. Must be a number.")
                        continue

                    gender = input(f"Gender [{patient.get('gender')}]: ").strip() or patient.get('gender')
                    blood_type = input(f"Blood Type [{patient.get('blood_type')}]: ").strip() or patient.get('blood_type')
                    address = input(f"Address [{patient.get('address')}]: ").strip() or patient.get('address')
                    phone = input(f"Phone [{patient.get('phone')}]: ").strip() or patient.get('phone')
                    email = input(f"Email [{patient.get('email')}]: ").strip() or patient.get('email')
                    insurance_provider_id = input(f"Insurance Provider ID [{patient.get('insurance_provider_id')}]: ").strip()
                    insurance_provider_id = int(insurance_provider_id) if insurance_provider_id.isdigit() else patient.get('insurance_provider_id')
                    insurance_policy_number = input(f"Insurance Policy Number [{patient.get('insurance_policy_number')}]: ").strip() or patient.get('insurance_policy_number')
                    primary_physician = input(f"Primary Physician [{patient.get('primary_physician')}]: ").strip() or patient.get('primary_physician')
                    emergency_contact = input(f"Emergency Contact [{patient.get('emergency_contact')}]: ").strip() or patient.get('emergency_contact')
                    emergency_phone = input(f"Emergency Phone [{patient.get('emergency_phone')}]: ").strip() or patient.get('emergency_phone')
                    medical_history = input(f"Medical History [{patient.get('medical_history')}]: ").strip() or patient.get('medical_history')
                    current_medications = input(f"Current Medications [{patient.get('current_medications')}]: ").strip() or patient.get('current_medications')
                    allergies = input(f"Allergies [{patient.get('allergies')}]: ").strip() or patient.get('allergies')
                    disease = input(f"Disease/Condition [{patient.get('disease')}]: ").strip() or patient.get('disease')

                    update_patient(
                        patient_id=patient_id,
                        name=name,
                        age=age,
                        gender=gender,
                        blood_type=blood_type,
                        address=address,
                        phone=phone,
                        email=email,
                        insurance_provider_id=insurance_provider_id,
                        insurance_policy_number=insurance_policy_number,
                        primary_physician=primary_physician,
                        emergency_contact=emergency_contact,
                        emergency_phone=emergency_phone,
                        medical_history=medical_history,
                        current_medications=current_medications,
                        allergies=allergies,
                        disease=disease
                    )

                    print("Patient information updated successfully.")

            except Exception as e:
                print(f"Error retrieving or updating patient data: {e}")
            finally:
                connection.close()
            
        elif choice == '3':
            view_patients()
            
        elif choice == '4':
            search_term = input("Enter search term (name, phone, or ID): ")
            # You'll need to implement search_patient() function
            print("Search functionality to be implemented")
            
        elif choice == '5':
            patient_id = int(input("Enter patient ID to delete: "))
            confirm = input(f"Are you sure you want to delete patient ID {patient_id}? (y/n): ")
            if confirm.lower() == 'y':
                # You'll need to implement delete_patient() function
                print("Delete functionality to be implemented")
                
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please try again.")

def doctor_management_menu():
    """Display doctor management menu and handle user input"""
    while True:
        print("\n=== DOCTOR MANAGEMENT ===")
        print("1. Add New Doctor")
        print("2. Update Doctor Information")
        print("3. View All Doctors")
        print("4. Search Doctor")
        print("5. Delete Doctor")
        print("6. Back to Main Menu")
        
        choice = input("Enter your choice (1-6): ")
        
        if choice == '1':
            name = input("Enter doctor name: ")
            specialization = input("Enter specialization: ")
            department_id = input("Enter department ID (optional): ")
            department_id = int(department_id) if department_id.isdigit() else None
            qualification = input("Enter qualification (optional): ")
            years_of_experience = input("Enter years of experience (optional, number): ")
            years_of_experience = int(years_of_experience) if years_of_experience and years_of_experience.isdigit() else None
            phone = input("Enter phone (optional): ")
            email = input("Enter email (optional): ")
            consultation_fee = float(input("Enter consultation fee: "))
            availability = input("Enter availability details (optional): ")
            bio = input("Enter bio (optional): ")
            
            add_doctor(
                name=name,
                specialization=specialization,
                consultation_fee=consultation_fee,
                department_id=department_id,
                qualification=qualification if qualification else None,
                years_of_experience=years_of_experience,
                phone=phone if phone else None,
                email=email if email else None,
                availability=availability if availability else None,
                bio=bio if bio else None
            )
            
        elif choice == '2':
            doctor_id = int(input("Enter doctor ID to update: "))
            # Get current doctor data
            connection = get_db_connection()
            try:
                with connection.cursor() as cursor:
                    sql = "SELECT * FROM doctor WHERE doctor_id = %s"
                    cursor.execute(sql, (doctor_id,))
                    doctor = cursor.fetchone()
                    
                    if not doctor:
                        print("Doctor not found!")
                        continue
                        
                    print("\nCurrent Doctor Information:")
                    for key, value in doctor.items():
                        print(f"{key}: {value}")
                    
                    print("\nEnter new information (leave blank to keep current value):")
                    name = input(f"Name [{doctor['name']}]: ") or doctor['name']
                    specialization = input(f"Specialization [{doctor['specialization']}]: ") or doctor['specialization']
                    department_id = input(f"Department ID [{doctor.get('department_id', '')}]: ")
                    department_id = int(department_id) if department_id.isdigit() else doctor.get('department_id')
                    qualification = input(f"Qualification [{doctor.get('qualification', '')}]: ") or doctor.get('qualification')
                    years_of_experience = input(f"Years of Experience [{doctor.get('years_of_experience', '')}]: ")
                    years_of_experience = int(years_of_experience) if years_of_experience.isdigit() else doctor.get('years_of_experience')
                    phone = input(f"Phone [{doctor.get('phone', '')}]: ") or doctor.get('phone')
                    email = input(f"Email [{doctor.get('email', '')}]: ") or doctor.get('email')
                    consultation_fee = input(f"Consultation Fee [{doctor['consultation_fee']}]: ") or doctor['consultation_fee']
                    availability = input(f"Availability [{doctor.get('availability', '')}]: ") or doctor.get('availability')
                    bio = input(f"Bio [{doctor.get('bio', '')}]: ") or doctor.get('bio')
                    
                    # Update the doctor in the database
                    update_doctor(
                        doctor_id=doctor_id,
                        name=name,
                        specialization=specialization,
                        department_id=department_id,
                        qualification=qualification,
                        years_of_experience=years_of_experience,
                        phone=phone,
                        email=email,
                        consultation_fee=consultation_fee,
                        availability=availability,
                        bio=bio
                    )
                    
            except Exception as e:
                print(f"Error retrieving doctor data: {e}")
            finally:
                connection.close()
                
        elif choice == '3':
            view_doctors()
            
        elif choice == '4':
            search_term = input("Enter search term (name, specialization, or ID): ")
            # You'll need to implement search_doctor() function
            print("Search functionality to be implemented")
            
        elif choice == '5':
            doctor_id = int(input("Enter doctor ID to delete: "))
            confirm = input(f"Are you sure you want to delete doctor ID {doctor_id}? (y/n): ")
            if confirm.lower() == 'y':
                # You'll need to implement delete_doctor() function
                print("Delete functionality to be implemented")
                
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please try again.")

def appointment_management_menu():
    """Display appointment management menu and handle user input"""
    while True:
        print("\n=== APPOINTMENT MANAGEMENT ===")
        print("1. Schedule New Appointment")
        print("2. Update Appointment")
        print("3. View All Appointments")
        print("4. Search Appointments")
        print("5. Cancel Appointment")
        print("6. Back to Main Menu")
        
        choice = input("Enter your choice (1-6): ")
        
        if choice == '1':
            patient_id = int(input("Enter patient ID: "))
            doctor_id = int(input("Enter doctor ID: "))
            date = input("Enter appointment date (YYYY-MM-DD): ")
            time = input("Enter appointment time (HH:MM): ")
            duration = input("Enter duration in minutes (default 30): ")
            duration = int(duration) if duration.isdigit() else 30
            reason = input("Enter reason for appointment (optional): ")
            notes = input("Enter any notes (optional): ")
            
            schedule_appointment(
                patient_id=patient_id,
                doctor_id=doctor_id,
                date=date,
                time=time,
                duration=duration,
                reason=reason if reason else None,
                notes=notes if notes else None
            )
            
        elif choice == '2':
            appointment_id = int(input("Enter appointment ID to update: "))
            # Get current appointment data
            connection = get_db_connection()
            try:
                with connection.cursor() as cursor:
                    sql = "SELECT * FROM appointment WHERE appointment_id = %s"
                    cursor.execute(sql, (appointment_id,))
                    appointment = cursor.fetchone()
                    
                    if not appointment:
                        print("Appointment not found!")
                        continue
                        
                    print("\nCurrent Appointment Information:")
                    for key, value in appointment.items():
                        print(f"{key}: {value}")
                    
                    print("\nEnter new information (leave blank to keep current value):")
                    patient_id = input(f"Patient ID [{appointment['patient_id']}]: ") or appointment['patient_id']
                    doctor_id = input(f"Doctor ID [{appointment['doctor_id']}]: ") or appointment['doctor_id']
                    date = input(f"Date [{appointment['date']}] (YYYY-MM-DD): ") or appointment['date']
                    time = input(f"Time [{appointment['time']}] (HH:MM): ") or appointment['time']
                    duration = input(f"Duration [{appointment['duration']}]: ") or appointment['duration']
                    reason = input(f"Reason [{appointment.get('reason', '')}]: ") or appointment.get('reason')
                    notes = input(f"Notes [{appointment.get('notes', '')}]: ") or appointment.get('notes')
                    status = input(f"Status [{appointment['status']}]: ") or appointment['status']
                    follow_up_date = input(f"Follow-up Date [{appointment.get('follow_up_date', '')}] (YYYY-MM-DD, optional): ") or appointment.get('follow_up_date')
                    
                    # Update the appointment in the database
                    update_appointment_status(
                        appointment_id=appointment_id,
                        status=status,
                        follow_up_date=follow_up_date if follow_up_date else None
                    )
                    
                    print("Appointment updated successfully.")
                    
            except Exception as e:
                print(f"Error retrieving appointment data: {e}")
            finally:
                connection.close()
                
        elif choice == '3':
            view_appointments()
            
        elif choice == '4':
            search_term = input("Enter search term (patient name, doctor name, or date): ")
            # You'll need to implement search_appointments() function
            print("Search functionality to be implemented")
            
        elif choice == '5':
            appointment_id = int(input("Enter appointment ID to cancel: "))
            confirm = input(f"Are you sure you want to cancel appointment ID {appointment_id}? (y/n): ")
            if confirm.lower() == 'y':
                update_appointment_status(
                    appointment_id=appointment_id,
                    status='Cancelled'
                )
                print("Appointment cancelled successfully.")
                
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please try again.")

def billing_management_menu():
    """Display billing management menu and handle user input"""
    while True:
        print("\n=== BILLING MANAGEMENT ===")
        print("1. Generate New Bill")
        print("2. Update Bill Status")
        print("3. View All Bills")
        print("4. Search Bills")
        print("5. Back to Main Menu")
        
        choice = input("Enter your choice (1-5): ")
        
        if choice == '1':
            patient_id = int(input("Enter patient ID: "))
            doctor_id = int(input("Enter doctor ID: "))
            appointment_id = input("Enter appointment ID (optional, leave blank if none): ")
            if appointment_id.strip() == '':
                appointment_id = None
            else:
                appointment_id = int(appointment_id)
                
            amount = float(input("Enter amount: "))
            tax = float(input("Enter tax amount (default 0): ") or 0)
            discount = float(input("Enter discount amount (default 0): ") or 0)
            date = input("Enter billing date (YYYY-MM-DD, default today): ") or datetime.now().strftime('%Y-%m-%d')
            due_date = input("Enter due date (YYYY-MM-DD, default 30 days from today): ") or (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
            status = input("Enter status (Paid/Unpaid/Partial/Overdue, default Unpaid): ") or 'Unpaid'
            payment_method = input("Enter payment method (Cash/Credit Card/Debit Card/Insurance/Bank Transfer/Other, optional): ")
            payment_details = input("Enter payment details (optional): ")
            items = input("Enter billed items as JSON (optional): ")
            
            generate_bill(
                patient_id=patient_id,
                doctor_id=doctor_id,
                appointment_id=appointment_id,
                amount=amount,
                tax=tax,
                discount=discount,
                date=date,
                due_date=due_date,
                status=status,
                payment_method=payment_method if payment_method else None,
                payment_details=payment_details if payment_details else None,
                items=items if items else None
            )
            
        elif choice == '2':
            bill_id = int(input("Enter bill ID to update: "))
            new_status = input("Enter new status (Paid/Unpaid/Partial/Overdue): ")
            payment_method = input("Enter payment method (optional): ")
            payment_details = input("Enter payment details (optional): ")
            
            update_bill_status(
                bill_id=bill_id,
                status=new_status,
                payment_method=payment_method if payment_method else None,
                payment_details=payment_details if payment_details else None
            )
            print("Bill status updated successfully.")
                
        elif choice == '3':
            view_bills()
            
        elif choice == '4':
            search_term = input("Enter search term (patient name, bill ID, or status): ")
            # You'll need to implement search_bills() function
            print("Search functionality to be implemented")
                
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

def medical_records_menu():
    """Display medical records management menu and handle user input"""
    while True:
        print("\n=== MEDICAL RECORDS ===")
        print("1. Add New Medical Record")
        print("2. Update Medical Record")
        print("3. View All Medical Records")
        print("4. Search Medical Records")
        print("5. Export Medical Records")
        print("6. Back to Main Menu")
        
        choice = input("Enter your choice (1-6): ")
        
        if choice == '1':
            patient_id = int(input("Enter patient ID: "))
            doctor_id = input("Enter doctor ID (optional, leave blank if none): ")
            if doctor_id.strip() == '':
                doctor_id = None
            else:
                doctor_id = int(doctor_id)
                
            visit_type = input("Enter visit type (Routine Checkup/Emergency/Follow-up/Specialist Consultation/Other): ")
            diagnosis = input("Enter diagnosis: ")
            symptoms = input("Enter symptoms (optional): ")
            treatment = input("Enter treatment details (optional): ")
            prescription = input("Enter prescription details (optional): ")
            tests_ordered = input("Enter tests ordered (optional): ")
            test_results = input("Enter test results (optional): ")
            notes = input("Enter any additional notes (optional): ")
            follow_up_required = input("Is follow-up required? (yes/no, default no): ").lower() == 'yes'
            follow_up_date = None
            if follow_up_required:
                follow_up_date = input("Enter follow-up date (YYYY-MM-DD): ")
            record_date = input("Enter record date (YYYY-MM-DD, default today): ") or datetime.now().strftime('%Y-%m-%d')
            
            add_medical_record(
                patient_id=patient_id,
                doctor_id=doctor_id,
                visit_type=visit_type,
                diagnosis=diagnosis,
                symptoms=symptoms if symptoms else None,
                treatment=treatment if treatment else None,
                prescription=prescription if prescription else None,
                tests_ordered=tests_ordered if tests_ordered else None,
                test_results=test_results if test_results else None,
                notes=notes if notes else None,
                follow_up_required=follow_up_required,
                follow_up_date=follow_up_date if follow_up_date else None,
                date=record_date
            )
            
        elif choice == '2':
            record_id = int(input("Enter medical record ID to update: "))
            # Get current record data
            connection = get_db_connection()
            try:
                with connection.cursor() as cursor:
                    sql = "SELECT * FROM medical_record WHERE record_id = %s"
                    cursor.execute(sql, (record_id,))
                    record = cursor.fetchone()
                    
                    if not record:
                        print("Medical record not found!")
                        continue
                        
                    print("\nCurrent Medical Record Information:")
                    for key, value in record.items():
                        print(f"{key}: {value}")
                    
                    print("\nEnter new information (leave blank to keep current value):")
                    patient_id = input(f"Patient ID [{record['patient_id']}]: ") or record['patient_id']
                    doctor_id = input(f"Doctor ID [{record.get('doctor_id', '')}]: ")
                    if doctor_id.strip() == '':
                        doctor_id = record.get('doctor_id')
                    else:
                        doctor_id = int(doctor_id)
                        
                    visit_type = input(f"Visit Type [{record['visit_type']}]: ") or record['visit_type']
                    diagnosis = input(f"Diagnosis [{record['diagnosis']}]: ") or record['diagnosis']
                    symptoms = input(f"Symptoms [{record.get('symptoms', '')}]: ") or record.get('symptoms')
                    treatment = input(f"Treatment [{record.get('treatment', '')}]: ") or record.get('treatment')
                    prescription = input(f"Prescription [{record.get('prescription', '')}]: ") or record.get('prescription')
                    tests_ordered = input(f"Tests Ordered [{record.get('tests_ordered', '')}]: ") or record.get('tests_ordered')
                    test_results = input(f"Test Results [{record.get('test_results', '')}]: ") or record.get('test_results')
                    notes = input(f"Notes [{record.get('notes', '')}]: ") or record.get('notes')
                    follow_up_required = input(f"Follow-up Required [{record['follow_up_required']}] (yes/no): ")
                    if follow_up_required.lower() in ('yes', 'no'):
                        follow_up_required = follow_up_required.lower() == 'yes'
                    else:
                        follow_up_required = record['follow_up_required']
                    follow_up_date = input(f"Follow-up Date [{record.get('follow_up_date', '')}] (YYYY-MM-DD): ") or record.get('follow_up_date')
                    record_date = input(f"Record Date [{record['date']}] (YYYY-MM-DD): ") or record['date']
                    
                    # Update the medical record in the database
                    update_medical_record(
                        record_id=record_id,
                        patient_id=patient_id,
                        doctor_id=doctor_id,
                        visit_type=visit_type,
                        diagnosis=diagnosis,
                        symptoms=symptoms,
                        treatment=treatment,
                        prescription=prescription,
                        tests_ordered=tests_ordered,
                        test_results=test_results,
                        notes=notes,
                        follow_up_required=follow_up_required,
                        follow_up_date=follow_up_date,
                        date=record_date
                    )
                    
                    print("Medical record updated successfully.")
                    
            except Exception as e:
                print(f"Error retrieving medical record data: {e}")
            finally:
                connection.close()
                
        elif choice == '3':
            view_medical_records()
            
        elif choice == '4':
            search_term = input("Enter search term (patient name, diagnosis, or doctor name): ")
            # You'll need to implement search_medical_records() function
            print("Search functionality to be implemented")
            
        elif choice == '5':
            print("\nExport Options:")
            print("1. Export to CSV")
            print("2. Export to Excel")
            print("3. Cancel")
            
            export_choice = input("Enter your choice (1-3): ")
            
            if export_choice == '1':
                filename = input("Enter filename (e.g., medical_records.csv): ")
                records = view_medical_records(export_csv=False, export_excel=False)
                if records is not None:
                    export_to_csv(records, filename)
            elif export_choice == '2':
                filename = input("Enter filename (e.g., medical_records.xlsx): ")
                records = view_medical_records(export_csv=False, export_excel=False)
                if records is not None:
                    export_to_excel(records, filename)
            elif export_choice == '3':
                continue
            else:
                print("Invalid choice.")
                
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please try again.")

def reports_menu():
    """Display reports menu and handle user input"""
    while True:
        print("\n=== REPORTS & ANALYTICS ===")
        print("1. Financial Reports")
        print("2. Operational Reports")
        print("3. Doctor Performance")
        print("4. Patient Statistics")
        print("5. Demographic Analysis")
        print("6. Appointment Patterns")
        print("7. Revenue Analysis")
        print("8. Back to Main Menu")
        
        choice = input("Enter your choice (1-8): ")
        
        if choice == '1':
            print("\nFinancial Report Time Frame:")
            print("1. Daily")
            print("2. Weekly")
            print("3. Monthly")
            print("4. Cancel")
            
            time_choice = input("Enter your choice (1-4): ")
            
            if time_choice == '1':
                generate_financial_report('day')
            elif time_choice == '2':
                generate_financial_report('week')
            elif time_choice == '3':
                generate_financial_report('month')
            elif time_choice == '4':
                continue
            else:
                print("Invalid choice.")
                
        elif choice == '2':
            print("\nOperational Report Time Frame:")
            print("1. Daily")
            print("2. Weekly")
            print("3. Monthly")
            print("4. Cancel")
            
            time_choice = input("Enter your choice (1-4): ")
            
            if time_choice == '1':
                generate_operational_report('day')
            elif time_choice == '2':
                generate_operational_report('week')
            elif time_choice == '3':
                generate_operational_report('month')
            elif time_choice == '4':
                continue
            else:
                print("Invalid choice.")
                
        elif choice == '3':
            generate_doctor_performance_report()
            
        elif choice == '4':
            generate_patient_statistics_report()
            
        elif choice == '5':
            analyze_patient_demographics()
            
        elif choice == '6':
            analyze_appointment_patterns()
            
        elif choice == '7':
            analyze_revenue_streams()
            
        elif choice == '8':
            break
        else:
            print("Invalid choice. Please try again.")

# --- New: Menu Functions ---

def department_management_menu():
    """Display department management menu and handle user input."""
    while True:
        print("\n=== DEPARTMENT MANAGEMENT ===")
        print("1. Add New Department")
        print("2. Update Department Information")
        print("3. View All Departments")
        print("4. Delete Department")
        print("5. Back to Main Menu")
        
        choice = input("Enter your choice (1-5): ")
        
        if choice == '1':
            name = input("Enter department name: ")
            head = input("Enter head of department (optional): ")
            phone = input("Enter phone number (optional): ")
            email = input("Enter email (optional): ")
            add_department(name, head if head else None, phone if phone else None, email if email else None)
        elif choice == '2':
            dept_id = int(input("Enter department ID to update: "))
            name = input("Enter new name (leave blank to skip): ") or None
            head = input("Enter new head (leave blank to skip): ") or None
            phone = input("Enter new phone (leave blank to skip): ") or None
            email = input("Enter new email (leave blank to skip): ") or None
            update_department(dept_id, name, head, phone, email)
        elif choice == '3':
            view_departments()
        elif choice == '4':
            dept_id = int(input("Enter department ID to delete: "))
            confirm = input(f"Are you sure you want to delete department ID {dept_id}? (y/n): ")
            if confirm.lower() == 'y':
                delete_department(dept_id)
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

def staff_management_menu():
    """Display staff management menu and handle user input."""
    while True:
        print("\n=== STAFF MANAGEMENT ===")
        print("1. Add New Staff Member")
        print("2. Update Staff Information")
        print("3. View All Staff")
        print("4. Delete Staff Member")
        print("5. Back to Main Menu")
        
        choice = input("Enter your choice (1-5): ")
        
        if choice == '1':
            name = input("Enter staff member's name: ")
            role = input("Enter staff role (e.g., Nurse, Receptionist): ")
            dept_id = input("Enter department ID: ")
            dept_id = int(dept_id) if dept_id.isdigit() else None
            phone = input("Enter phone number (optional): ")
            email = input("Enter email (optional): ")
            address = input("Enter address (optional): ")
            add_staff(name, role, dept_id, phone if phone else None, email if email else None, address if address else None)
        elif choice == '2':
            staff_id = int(input("Enter staff ID to update: "))
            name = input("Enter new name (leave blank to skip): ") or None
            role = input("Enter new role (leave blank to skip): ") or None
            dept_id = input("Enter new department ID (leave blank to skip): ")
            dept_id = int(dept_id) if dept_id.isdigit() else None
            phone = input("Enter new phone (leave blank to skip): ") or None
            email = input("Enter new email (leave blank to skip): ") or None
            address = input("Enter new address (leave blank to skip): ") or None
            update_staff(staff_id, name, role, dept_id, phone, email, address)
        elif choice == '3':
            view_staff()
        elif choice == '4':
            staff_id = int(input("Enter staff ID to delete: "))
            confirm = input(f"Are you sure you want to delete staff member with ID {staff_id}? (y/n): ")
            if confirm.lower() == 'y':
                delete_staff(staff_id)
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

def insurance_management_menu():
    """Display insurance management menu and handle user input."""
    while True:
        print("\n=== INSURANCE MANAGEMENT ===")
        print("1. Add New Insurance Provider")
        print("2. Update Insurance Provider Info")
        print("3. View All Insurance Providers")
        print("4. Delete Insurance Provider")
        print("5. Back to Main Menu")
        
        choice = input("Enter your choice (1-5): ")
        
        if choice == '1':
            name = input("Enter provider name: ")
            contact = input("Enter contact person (optional): ")
            phone = input("Enter phone number (optional): ")
            email = input("Enter email (optional): ")
            address = input("Enter address (optional): ")
            add_insurance_provider(name, contact if contact else None, phone if phone else None, email if email else None, address if address else None)
        elif choice == '2':
            provider_id = int(input("Enter provider ID to update: "))
            name = input("Enter new name (leave blank to skip): ") or None
            contact = input("Enter new contact person (leave blank to skip): ") or None
            phone = input("Enter new phone (leave blank to skip): ") or None
            email = input("Enter new email (leave blank to skip): ") or None
            address = input("Enter new address (leave blank to skip): ") or None
            update_insurance_provider(provider_id, name, contact, phone, email, address)
        elif choice == '3':
            view_insurance_providers()
        elif choice == '4':
            provider_id = int(input("Enter provider ID to delete: "))
            confirm = input(f"Are you sure you want to delete provider with ID {provider_id}? (y/n): ")
            if confirm.lower() == 'y':
                delete_insurance_provider(provider_id)
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

def test_management_menu():
    """Display test management menu and handle user input."""
    while True:
        print("\n=== TEST MANAGEMENT ===")
        print("1. Add New Test Type")
        print("2. Order a Test for a Patient")
        print("3. Update a Patient's Test Results")
        print("4. View All Patient Test Records")
        print("5. Back to Main Menu")
        
        choice = input("Enter your choice (1-5): ")
        
        if choice == '1':
            name = input("Enter test name: ")
            cost = float(input("Enter test cost: "))
            description = input("Enter test description (optional): ")
            add_test_type(name, cost, description if description else None)
        elif choice == '2':
            patient_id = int(input("Enter patient ID: "))
            test_id = int(input("Enter test type ID: "))
            doctor_id = int(input("Enter ordering doctor ID: "))
            add_patient_test(patient_id, test_id, doctor_id)
        elif choice == '3':
            patient_test_id = int(input("Enter patient test record ID to update: "))
            results = input("Enter test results: ") or None
            date_completed = input("Enter date completed (YYYY-MM-DD, optional): ") or None
            status = input("Enter new status (Ordered/InProgress/Completed/Cancelled, optional): ") or None
            update_patient_test(patient_test_id, date_completed, results, status)
        elif choice == '4':
            view_patient_tests()
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

# --- New: Inventory Management Menu ---
def inventory_management_menu():
    """Display inventory management menu and handle user input."""
    while True:
        print("\n=== INVENTORY MANAGEMENT ===")
        print("1. Add New Inventory Item")
        print("2. Update Inventory Item")
        print("3. View All Inventory")
        print("4. Check Low Stock Items")
        print("5. Delete Inventory Item")
        print("6. Back to Main Menu")
        
        choice = input("Enter your choice (1-6): ")
        
        if choice == '1':
            name = input("Enter item name: ")
            category = input("Enter category: ")
            quantity = int(input("Enter initial quantity: "))
            unit = input("Enter unit (e.g., box, vial): ")
            price = float(input("Enter price per unit: "))
            supplier = input("Enter supplier (optional): ") or None
            expiry_date = input("Enter expiry date (YYYY-MM-DD, optional): ") or None
            threshold = input("Enter reorder threshold (optional, default 0): ")
            threshold = int(threshold) if threshold.isdigit() else 0
            description = input("Enter description (optional): ") or None
            last_restocked = datetime.now().strftime('%Y-%m-%d')
            add_inventory_item(name, category, quantity, unit, price, supplier, expiry_date, threshold, last_restocked, description)
        
        elif choice == '2':
            item_id = int(input("Enter item ID to update: "))
            print("\nEnter new values (leave blank to keep current):")
            name = input("New name: ") or None
            category = input("New category: ") or None
            quantity = input("New quantity: ")
            quantity = int(quantity) if quantity.isdigit() else None
            unit = input("New unit: ") or None
            price = input("New price: ")
            price = float(price) if price else None
            supplier = input("New supplier: ") or None
            expiry_date = input("New expiry date (YYYY-MM-DD): ") or None
            threshold = input("New reorder threshold: ")
            threshold = int(threshold) if threshold.isdigit() else None
            description = input("New description: ") or None
            update_inventory_item(item_id, quantity, price, threshold, supplier, datetime.now().strftime('%Y-%m-%d'), expiry_date, name, category, unit, description)
        
        elif choice == '3':
            view_inventory()
            
        elif choice == '4':
            check_low_stock()
            
        elif choice == '5':
            item_id = int(input("Enter item ID to delete: "))
            confirm = input(f"Are you sure you want to delete item with ID {item_id}? (y/n): ")
            if confirm.lower() == 'y':
                delete_inventory_item(item_id)
        
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please try again.")

# Main menu function
def main_menu():
    """Display the main menu and handle user input"""
    while True:
        print("\n=== HOSPITAL MANAGEMENT SYSTEM ===")
        print("1. Patient Management")
        print("2. Doctor Management")
        print("3. Appointment Management")
        print("4. Billing Management")
        print("5. Medical Records")
        print("6. Reports and Analytics")
        print("7. Department Management")
        print("8. Staff Management")
        print("9. Insurance Management")
        print("10. Test Management")
        print("11. Inventory Management")
        print("12. Exit")
        
        choice = input("Enter your choice (1-12): ")
        
        if choice == '1':
            patient_management_menu()
        elif choice == '2':
            doctor_management_menu()
        elif choice == '3':
            appointment_management_menu()
        elif choice == '4':
            billing_management_menu()
        elif choice == '5':
            medical_records_menu()
        elif choice == '6':
            reports_menu()
        elif choice == '7':
            department_management_menu()
        elif choice == '8':
            staff_management_menu()
        elif choice == '9':
            insurance_management_menu()
        elif choice == '10':
            test_management_menu()
        elif choice == '11':
            inventory_management_menu()
        elif choice == '12':
            print("Exiting Hospital Management System. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the main menu when the script is executed
if __name__ == "__main__":
    # Initialize the database connection
    init_db()
    
    # Start the application
    main_menu()

Error initializing database: (1826, "Duplicate foreign key constraint name 'fk_patient_insurance'")

=== HOSPITAL MANAGEMENT SYSTEM ===
1. Patient Management
2. Doctor Management
3. Appointment Management
4. Billing Management
5. Medical Records
6. Reports and Analytics
7. Department Management
8. Staff Management
9. Insurance Management
10. Test Management
11. Inventory Management
12. Exit
Exiting Hospital Management System. Goodbye!
